In [ ]:
import os
import re
import pandas as pd
import urllib
from nltk import SnowballStemmer
from gensim.models import Word2Vec
#import langdetect
import tika
import time
from tika import parser
import pickle

# Raw Articles pdf Data
Convert pdf to text using Tika apache server 

In [ ]:
# specify the target data folder
target_dir = 'C:/Thesis/Data/Journal Of management_2000_2020'

In [ ]:
# prep pdf extraction
pdf_files = []
wrd_files = []

for f in os.listdir(target_dir):
    if f.endswith(".pdf") | f.endswith(".PDF"):
        thispdf = os.path.join(target_dir, f)
        pdf_files = pdf_files + [thispdf]
    if f.endswith(".doc") | f.endswith(".docx") | f.endswith(".DOC") | f.endswith(".DOCX"):
        thiswrd = os.path.join(target_dir, f)
        wrd_files = wrd_files + [thiswrd]

In [ ]:
port = 4321 # port to use for Tika server (chosen arbitrarily)
server_str = 'http://localhost:' + str(port)
tika_path  = 'C:\Software\ tika-server-1.23.jar'
tika_run   = 'java -jar' + tika_path + ' --port ' + str(port)
print('Command to start Tika:')
print(tika_run)
print('---')
print('Number of pdf files: ' + str(len(pdf_files)))
print('Number of word files: ' + str(len(wrd_files)))

# PDF Metadata

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

pdf_metadata = [parser.from_file(t, server_str, xmlContent=False)["metadata"] for t in (pdf_files + wrd_files)]

In [ ]:
# save data metadata
pickle.dump(pdf_metadata, open( "C:/Thesis/Data/save/journal_of_management/metadata/metadata_all.p", "wb" ) )

In [1]:
# load metadata
import pickle

pdf_metadata = pickle.load(open("C:/Thesis/Data/save/journal_of_management/metadata/metadata_all.p", "rb"))

In [3]:
#pdf_metadata

# pdf content

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

#pdf_text = [parser.from_file(t, server_str, xmlContent=False)["content"] for t in (pdf_files + wrd_files)]

# Load pdf content

In [4]:
import pickle
#load the text data
pdf_data = pickle.load(open("C:/Thesis/Data/save/journal_of_management/save_alltext_jm.p", "rb"))


# Get title, author and creator from metadata

In [5]:
pdf_title = []
pdf_Author = []
pdf_subject = []
#pdf_creator =[]
#pdf_Keywords = []

for i in pdf_metadata:
    try:
        temp = i['title']
        auth_temp = i['Author']
        subj_temp = i['subject']
       #cre_temp = i['creator']
        #key_temp = i['Keywords']
        
        
        pdf_title.append(temp)
        pdf_Author.append(auth_temp)
        pdf_subject.append(subj_temp)
        #pdf_creator.append(cre_temp)
        #pdf_Keywords.append(key_temp)
    except:
        #print("An exception occurred" )
        pdf_title.append(temp)
        pdf_Author.append(auth_temp)
        pdf_subject.append(subj_temp)
        #pdf_creator.append(cre_temp)
        #pdf_Keywords.append(key_temp)

In [30]:
pdf_subject[1]

'Journal of Management 2005.31:680-699'

In [8]:
import re
# step-1 preprocess subject
pdf_sub = []

  # pre-processing for subject
for i in pdf_subject:
    # remove special character
    t = (re.sub(r'[?\-|.|:|!]','',i))
    # remove digits
    t = (re.sub("\d+", "", t))
    # remove end whitespace
    t = re.sub(r"\s+$","",t)
    # append to the list
    pdf_sub.append(t)

In [10]:
# step-2 remove subject
pdf_txt_sub = []

for i in pdf_data:
    line = i
       # for subject
    for m in pdf_sub:
        line = re.sub(m, '', line)

        
    pdf_txt_sub.append(line)
        

In [11]:
# save removed subject 
pickle.dump(pdf_txt_sub, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject/metadata_all.p", "wb" ) )


In [23]:
# step 3 pre-process author
# for author
pdf_au = []

for i in pdf_txt_sub:
    line = i
    for j in pdf_Author:
        if ',' in j:
            k = re.sub(' and ', ' ', j)
            l = k.split(",")
            for r in l:
                line = re.sub(r.strip(), '', line)

        else:
            k = re.sub(' and ', ',', j)
            l = k.split(",")
            for r in l:
                line = re.sub(r.strip(), '', line)

    pdf_au.append(line)

In [26]:
#pdf_au[1]

In [24]:
# save removed subject + author
pickle.dump(pdf_au, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject_author/metadata_all.p", "wb" ) )


In [27]:
# step-4 remove title

pdf_text_title = []
for i in pdf_au:
    line = i
    for j in pdf_title:
         line = re.sub(j, '', line)
    pdf_text_title.append(line)

In [28]:
pdf_text_title[1]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n10.1177/0149206305279054ARTICLE / October 2005Arendt et al. / CEO-Adviser Model\n\nA CEO-Adviser Model\nof Strategic Decision Making†\n\n*\nProfessional Programs in Business, University of Wisconsin–Green Bay,\n\n2420 Nicolet Drive, WH 460, Green Bay, WI 54311\n\n\nSchool of Business Administration, Management & Organizations Area,\n\nUniversity of Wisconsin–Milwaukee, P.O. Box 742, Milwaukee, WI 53201\n\n\nBelk College of Business Administration, University of North Carolina at Charlotte,\n\n9201 University City Blvd., Charlotte, NC 28223\n\nUpper echelons research has emphasized decision making either by individual CEOs or by teams\nof top managers. The authors introduce the CEO-Adviser model as an intermediate model of stra-\ntegic decision making. The CEO-Adviser model leads to new propositions that have not been\nexplored through the individual CEO or top management team models conce

In [31]:
# save removed subject + author
pickle.dump(pdf_text_title, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_subject_author_title/metadata_all.p", "wb" ) )


In [ ]:
# remove title, author 
# import re

# pdf_text = []
# pdf_sub = []

# # pre-processing for subject
# for i in pdf_subject:
#     # remove special character
#     t = (re.sub(r'[?\-|.|:|!]','',i))
#     # remove digits
#     t = (re.sub("\d+", "", t))
#     # remove end whitespace
#     t = re.sub(r"\s+$","",t)
#     # append to the list
#     pdf_sub.append(t)

# # filter out for author, title and subject
# for i in pdf_data:
#     line = i
#     # for author
#     for j in pdf_Author:
#         if ',' in j:
#             k = re.sub(' and ', ' ', j)
#             l = k.split(",")
#             for r in l:
#                 line = re.sub(r.strip(), '', line)
#         else:
#             k = re.sub(' and ', ',', j)
#             l = k.split(",")
#             for r in l:
#                 line = re.sub(r.strip(), '', line)
                
#     # for title       
#     for j in pdf_title:
#          line = re.sub(j, '', line)

#     # for subject
#     for m in pdf_sub:
#         line = re.sub(m, '', line)

        
#     pdf_text.append(line)
        

# 2 -  Convert pdf text to lower case

In [37]:
#Convert each string to lowercase
for i in range(len(pdf_text_title)):
    pdf_text_title[i] = pdf_text_title[i].lower()

In [38]:
# remove references section 
import re
pdf_ref = [re.sub(r"(?is)\nreferences\n.+", "", f) for f in pdf_text_title]

In [ ]:
# not required
def eliminate_method_result(method, discussion):
    
    import re    
    start = method
    end = discussion
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]

    return pdf_method_dis

In [ ]:
# not required
def eliminate_method_result_ref(method, dis, ref):
    
    import re

    start = method
    end = dis
    
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]
    
    start_ref = ref
    pattern_ref = start_ref +'(.|\n)*$'

    pdf_ref= [re.sub(pattern_ref, '', f) for f in pdf_method_dis]
    
    return pdf_ref

# 3 - Pre-processing
Regular expression is used to clean up the pre-processed data and saved as a Master data

In [43]:
# Authors names are saved already (code is available at author name folder-get name list notebook)
#load authors names
import pickle
author_nam = pickle.load(open("C:/Thesis/Data/save/Master_Data/auth_ref_filter_num_char/auth_Journal_of_management.p", "rb"))

In [44]:
# GET REFERENCE LISTS OF STOPWORDS, I.E. WORDS THAT ARE ESSENTIALLY MEANINGLESS
# Get all the list of authors names

from nltk.corpus import stopwords

sw = stopwords.words('english') + author_nam
#print(sw)

In [11]:
# Function to filter out the stopwords and authors names
def filterWords(msg, words):
    m = msg.split(' ')
    words_end = [w + '.' for w in words]
    filtered_words = list(filter(lambda word: word not in words, m))
    filtered_words = [(w if w not in words_end else '.') for w in filtered_words]
    result = ' '.join(filtered_words)
    return(result)

In [ ]:
# GET RID OF THE STOPWORDS IN TEXTS --running
data_proc_1 = [filterWords(f, sw) for f in pdf_p]

In [48]:
# save data
pickle.dump(data_proc_1, open( "C:/Thesis/Data/save/journal_of_management/afterfilter/data_proc_1.p", "wb" ) )

In [1]:
import pickle
#load the text data
data_proc_1 = pickle.load(open("C:/Thesis/Data/save/journal_of_management/afterfilter/data_proc_1.p", "rb"))

In [2]:
data_proc_1[122]

' antitakeover provisions shareholder value implications review contingency framework paper reviews literatures two competing theoretical views antitakeover provisions shareholder value relationship managerial entrenchment shareholder interests offers contingency framework reconcile views. framework based broader agency theory perspective organizational literatures power incorporates two key moderators corporate shareholder monitoring draws attention rather whether antitakeover provisions enhance erode shareholder value.  elsevier science . rights reserved. managerial responses active takeover market sparked considerable controversy. common strategic action taken managers even prior takeover bid adoption antitakeover provisions essentially involves company corporate charter operating policy makes takeover expensive. revival merger acquisition activity reactivated controversy additional adopting antitakeover provisions . moreover powerful shareholders pension funds increasingly challeng

In [5]:
import re

pdf_p= [re.sub(r"\n(\n)*", "\n", t)for t in data_proc_1]

pdf_p= [re.sub(r"\n", " ", t)for t in pdf_p]
# remove text with [] eg citation
pdf_p = [re.sub(r"\[[^)]*\]", "", t)for t in pdf_p]
#Remove numbers
pdf_p = [re.sub('[0-9]+', '', f) for f in pdf_p]
#pdf_p = [re.sub("r[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?", " ", t)for t in pdf_p]
pdf_p = [re.sub('[!@/%“”‘:#©β<>+=δχ*&$]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('’s','', f) for f in pdf_p]
pdf_p = [re.sub('’re','', f) for f in pdf_p]
pdf_p = [re.sub('’t','', f) for f in pdf_p]
pdf_p = [re.sub('’ve','', f) for f in pdf_p]
pdf_p = [re.sub('’ll','', f) for f in pdf_p]
pdf_p = [re.sub('’m','', f) for f in pdf_p]

pdf_p = [re.sub('[:()-]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('[\.!?]+ ', 'XYZXYZ', t) for t in pdf_p]
# \w will match alphanumeric characters and underscores
# [^\w] will match anything that's not alphanumeric or underscore
pdf_p = [re.sub(r'[^\w]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('XYZXYZ', '.', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
# Replace multiple dots with one dot
pdf_p = [re.sub('\.\.+',".", f) for f in pdf_p]
pdf_p = [re.sub(r'\b\w{1,3}\b',"", f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]

pdf_p = [re.sub("^\s+","", f) for f in pdf_p]# remove the front space
pdf_p = [re.sub("\s+\Z","", f) for f in pdf_p]#remove the back space

In [7]:
author_list =  ["guerci", "longoni", "luzzini", "’donohue", "torugsa", 'bspba']
author_list += ["robert", "ackrill", "valerie", "caven", "jamila", "alaktif", 'alberti', 'gdanaj', 'design']
author_list += ['unit', 'london', 'school', 'economics', 'iouri', 'alexachine', 'joseph', 'amoah', 'siebert', 'manu', 'greg', 'bamber']
author_list += ['foobar', 'admin', 'bradford', 'bell', 'adam', 'kanar', 'steve', 'kozlowski', 'mousumi', 'bhattacharyapatrick']
author_list += ['wright', 'blair', 'tanya', 'bondarouk', "tanya", "bondarouk", "emma", "parry", "elfi", "furtmueller"]
author_list += ["bondarouk", "tanya", "trullen", "jordi", "valverde", "mireia", "corine", "boon", "rory", "eckardt", "david"]
author_list += ["lepak", "paul", "boselie", "anna", "nehles", "andre", "veenendaal", 'anna', 'nehles', 'nikos', 'bozionelos']
author_list += ['giorgos', 'konstantinos', 'kostopoulos', 'shyong', 'chwen', 'huey', 'yehuda', 'baruch', 'wenxia', 'zhou', 'bruce']
author_list += ['layla', 'jayne', 'branicki', 'brooks', 'varisli', 'usuario', 'joost', 'bücker', 'ziming', 'chakrabarti']
author_list += ['barnes', 'berthon', 'pitt', 'lmonkhouse', 'ucsm', 'charleston', 'gajewska', 'mattos', 'hchapman', 'sarjana', 'skolterman']
author_list += ['donna', 'chrobot', 'mason', 'marian', 'ruderman', 'todd', 'weber', 'patricia']
author_list += ['ohlott', 'maxine', 'dalton', 'david', 'collings', 'conrad', "harald", "conrad", "hendrik", 'meyer']
author_list += ["ohle", 'cook', 'mackenzie', 'rforde', 'coyne', 'farley', 'axtell', 'sprigg', 'best', 'lkwok']
author_list += ["marian", 'crowley', "henry", "akram", "ariss", 'school', 'management', 'miguel', 'pina', 'cunha', 'daverth', 'hyde'] 
author_list += ['pcassell', 'reuver', "renee", 'voorde', "karina", "kilroy", "steven"]
author_list += ['giovanni', 'stefano', "pauline", "dibben", "chris", "brewster", "michael", "brookes", "rita", "cunha"]
author_list += ["edward", "webster", "geoffrey", "wood", 'chris', 'brewster', 'unknown', 'tony', 'edwardssarosh', 'kuruvilla']
author_list += ['ehnert', 'jeff', 'ericksenlee', 'dyer', 'anonymous', 'alan', 'felstead', 'alan', 'felstead', 'duncan', 'gallie']
author_list += ['francis', 'green', 'ying', 'maria', 'fleischmann', 'luke', 'fletcher', 'xiaoni', 'computer', 'christian', 'grund']
author_list += ['christian', 'grund', 'johannes', 'martin', 'jane', 'nolan', 'marco', 'guerci', 'annachiara', 'longoni']
author_list += ['davide', 'luzzini', 'nina', 'lynette', 'harris', "michael", "harvey", "marina", "dabic", "kiessling", "jane"]
author_list += ["maley", "miriam", "moeller", 'marco', 'hauptmeier', 'tansley', 'carole', 'hytti', 'ulla', 'kautonen']
author_list += ['teemu', 'akola', 'elisa', "jones", 'derek', "kalmi", "panu", "kato", "takao", "mäkinen", "mikko"]
author_list += ["jonsen", "karsten", "point", "sébastien", "kelan", 'elisabeth', "grieble", "adrian", "jooss", "stefan"]
author_list += ["burbach", "ralf", "ruël", "huub", "jooss", "stefan", "mcdonnell", "anthony", "burbach", "ralf", "simy", "annilee", "game"]
author_list += ["ishita", "toshniwal", "yuhee", "jung", "norihiko", "takeuchi", "dong"]
author_list += ['kwan', "jung", "ruth", "aguilera", "michel", "goyer", "frances", "jørgensen", "karen", "becker"]
author_list += ["ulrich", "jürgens", "martin", "krzywdzinski", 'marta', 'kahancovámarc', 'meer', "pavithra", "kailasapathy", "jayakody"]
author_list += ["pavithra", "kailasapathy", "maria", "kraimer", "isabel", "metz", "kamoche"]
author_list += ["lisa", 'qixun', "siebers", 'patrick', 'kampkötter', "patrick", "kampkötter", "christine", "harbring", "dirk"]
author_list += ["sliwka", "haiying", "kang", "shen", "haiying", "kang", "shen", "john", "benson"]
author_list += ['kanstrén', 'kaisu', "ilias", "kapoutsis", "alexandros", "papalexandris", "ioannis", "thanos", "charlotte"]
author_list += ["karam", "fida", "afiouni", 'norifumi', 'kawai', "norifumi", "kawai", "alexander", "mohr", "norifumi"]
author_list += ["kawai", "roger", "strange", "ashlea", "kellner", "keith", "townsend", "adrian", "wilkinson", "regina"]
author_list += ["kempen", "barbara", "pangert", "kate", "hattrup", "karsten", "mueller", "ingela", "joens", "julia"]
author_list += ["kensbock", "stephan", "boehm", "peter", "kesting", "lynda", 'jiwen', "song", "zhihua", "michal", "krol"]
author_list += ['compaq', "zaheer", "khan", "rekha", "nicholson", "pervaiz", "akhtar", "shlomo", "tarba", "mohammad"]
author_list += ["ahammad", "vorley", 'pradip', 'khandwalla', "tatiana", "khvatova", "madeleine", "block", 'sehoon']
author_list += ['hyun', "minseo", "beehr", 'terry', "sunghoon", "sunwook", "chung", "hyondong", "yaping", "gong", "dong"]
author_list += ["dong", 'yeol', "zhiqiang", "kyungmin","moon", 'hyoung', "soyeon", "mannsoo", "shin", 'yeol']
author_list += ["samuel", "aryee", "raymond", "sang","hyondong", "jong", 'keon', "sanghyeon", "sung"]
author_list += ["kwang", 'hyun', "heejung", "byun", "andrea", "kyongji", "yongguen", "hyondong", "youngsang", 'lyong', 'josie']
author_list += ['kinge', 'susan', 'kirk', "andrea", 'kirk', "brown", "pieter", "dijk", "dirk", "klimkeit", "markus"]
author_list += ["reihlen", 'marlies', 'kluike', "deborah", 'erdos', "knapp", "cathy", "dubois", "mary", "hogue", "astakhova"]
author_list += ["robert", "faley", "knies", "boselie", "julian", 'gould', "williams", "wouter", "vandenabeele", "knies", "peter", "leisink"]
author_list += ["rens", "schoot", "angela", "knox", "warhurst", "dennis", "nickson", "dutton", "gordon", "young", 'erdogan', "satu"] 
author_list += ["koivisto", "ronald", "rice"]
author_list += ['constantine', 'kontoghiorghes', "dorien", "kooij", "jansen", "josje", "dikkers", "annet", "lange", "andreas"]
author_list += ["kornelakis", "michail", "veliziotis", "horen", "voskeritsian", "bernice", "kotey", "bishnu", "sharma"]
author_list += ["tina", "kowalski", "wendy", "loretto", "tina", "kowalski", "wendy", "loretto", "redman"]
author_list += ['robin', 'kramar', "claudia", "kröll", "stephan", "nüesch", "claudia", "kröll", "stephan", "nüesch", 'nils']
author_list += ["foege", "amadeus", "kubicek", "ramudu", "bhanugopan", "grant", "’neill", "kuipers", "laura"]
author_list += ["giurge", 'mukta', 'kulkarni', "mukta", "kulkarni", "caren", "rodrigues", "mukta", "kulkarni", "hugh"]
author_list += ["scullion", "kumar", "shamala", "hansiya", 'abdul', 'rauf', "rathnasekara", "hasara", "chien", 'chih', "kirk"]
author_list += ["chang", "sarah", "quinton", "chiu", "iling", "bård", "kuvaas", "lynn", "shore", "buch", "anders", "dysvik"]
author_list += ['soon', 'kwon', 'anastasia', 'kynighou', "marja", "känsälä", "liisa", "mäkelä", "vesa", "suutari"]
author_list += ['thomas', 'köllen', "jana", "kühnel", 'vahle', "hinz", "jessica", "bloom", "syrek", "esteban", "lafuente"]
author_list += ["jasmina", 'berbegal', "mirabent", "cristiana", "lages", "nigel", "piercy", "neeru", "malhotra", "cláudia"]
author_list += ["simões", "jennifer", "cheris", "chow", "raymond", 'lakshman', "jason", "lambert"]
author_list += ["dynah", "basuil", "myrtle", "bell", "marquardt", "lamberti", "giuseppe", "aluja", 'banet', "tomas", "rialp"]
author_list += ['criado', "josep", "langmann", "sten", "thomas", 'stefan', "robert", "émilie", "lapointe", "christian"]
author_list += ["vandenberghe", 'marina', 'latukha', "jakob", "lauring", "selmer", "annamaria", "kubovcikova", "hyun", 'jung', "carol"]
author_list += ["reade", "jean", "feng", "chan", "hyuneung", "jaeeun", "jongsun", "park"]
author_list += ["jeoung", "seongmin", "jingoo", "kang", 'kyung', "woonki", "hong", "ariel"]
author_list += ["avgar", "byron", "jing", "wang", "johanna", "weststar", "khan", 'yeon']
author_list += ['young', "jean", "flora", "chiang", "emmy", "esch", "zhenyao", 'administrator', "lejeune", "christophe"]
author_list += ["beausaert", "simon", "raemdonck", "isabel", "lieve", "lembrechts", "patrizia", "zanoni", "marijke"]
author_list += ["verbruggen", "tomasz", "lenartowicz", "james", "johnson", "konopaske", "dave", "lepak", "michael"]
author_list += ["dickmann", "emma", "parry", "levy", "orly", "jonsen", "peiperl", "maury", "bouquet", "cyril", "humbert"]
author_list += ['anne', "laure", "carmen", 'clark', "lewis", "ryan", "león", 'federico', "garcía", 'saavedra']
author_list += ['josé', "luis", 'daud', "xiaobei", "stephen", "frenkel", "yong", 'quan', "chih", 'hsing']
author_list += ["stella", "paul", "nesbit", "yongqiang", "guanglei", "zhang", "yang", 'huai', 'liang', 'liang', "liao", "xiang", "jiang"] 
author_list += ["xiao", 'song', "jing", "qian", "zhen", 'xiong', "chen", "yuntao"]
author_list += ["changwei", "dong", "xiyuan", "hsin", 'chen', "patrick", "bruning", "nina", "cole", "douglas", "flint"]
author_list += ["chanrith", "ngin", "vivien", "supangco", "colin", "lindsay", "johanna", "commander", "patricia", "findlay"]
author_list += ["marion", "bennie", 'dunlop', "corcoran", "meer", "sarah", "lindsay", "cathy", "sheehan", "helen"]
author_list += ["cieri", 'chih', 'hsing', 'nien', "ting", "heng", "chih", 'hsing']
author_list += ["bernard", 'nanfeng', "yucheng", "zhang", 'kwong', "kwan", "long", 'zeng', "xiao", 'ming', "zheng", "songbo"]
author_list += ["kaifeng", "jiang", "jianxun", "chen", "jingzhou", "xiaoshuang", "raymond", "hang", "kathleen", "keeling", 'nadia']
author_list += ["papamichail", "gordon", "rong", "chao", "yang", "xiaolang", "qihai", "huang", "hongli", "wang"]
author_list += ["shanshi", "yipeng", 'cooper', "cary", 'tarba', "shlomo", "richard", "long", "tony"]
author_list += ["fang", "mark", "loon", "lilian", 'otaye', "ebede", "stewart", "mary", "love", "susan"]
author_list += ["dustin", "xander", 'matthijs', "blomme", "rené", "schalk", "luiz", 'john', "spicer", "diana", "biao"]
author_list += ["shanshan", "zheng", "hongmei", "liang", "liang", "zhike", "ting", "anna", 'maija', "lämsä"]
author_list += ["suvi", "heikkinen", "mark", "smith", "christelle", "tornikoski", "sergio", 'lópez', "bohle", "pedro"]
author_list += ["leiva", "antonio", 'mladinic', "alonso", 'andrés', "maría", 'josé', "chambel", "antonia", 'diaz', 'valdes']
author_list += ["iriarte", "cristina", "duarte", "marta", 'vidal', "suárez", "belén", 'gonzález', "díaz", 'kristina']
author_list += ['løkke', "shaozhuang", 'gabriela', "silva", "victor", "callan", "virginia", "trigo", "charmein", "madden"]
author_list += ["jennifer", "ceyda", "maden", "hayat", "kabasakal", 'maden', 'eyiusta', 'ceyda', "jasmin", "mahadevan", "katharina"]
author_list += ['kilian', "yasin", "asri", "maharani", "gindo", "tampubolon", "upam", 'pushpak']
author_list += ["makhecha", "vasanthi", "srinivasan", "ganesh", "prabhu", "sourav", "mukherji", 'paula', 'makkonen', "marlin"]
author_list += ["malek", "pawan", "budhwar", 'sebastian', "reiche", "parul", "malik", "pooja", "garg", "ashish", "malik", "vijay", "pereira"] 
author_list += ["shlomo", "tarba", "miki", "malul", "amir", "shoham", "moshfique"]
author_list += ["uddin", "aminu", "mamman", "khamis", 'zayid', "kulaiby", "aminu", "mamman", "yudi", "somantri"]
author_list += ["sari", "mansour", "diane", 'gabrielle', "tremblay", "cassia", "marchon", "hugo", "toledo", "marcoux"]
author_list += ["gilles", "guihur", "izold", "leclerc", "andré", "joana", "marinova", "janneke", "plantenga"]
author_list += ["chantal", "remery", "raymond", "markey", "joseph", "mcivor", "chris", "wright", "janet", "marler", "john", "boudreau"]
author_list += ["janet", "marler", "emma", "parry", "katty", "marmenout", "pamela", "lirio"]
author_list += ["angela", "martin", "jackie", "wellen", "martin", "grimmer", "graeme", "martin", "sabina"]
author_list += ["siebert", "robson", "martini", "mattia", "cavenago", "dario", "marafioti", "elisabetta", "miguel"]
author_list += ['martínez', "lucio", "robert", "mackenzie", "daniele", "mascia", "silvia", 'dello', "russo", "federica"]
author_list += ["morandi", "munir", 'shehu', "mashi", "chandrakantan", "subramaniam", "johanim", "johari", 'makoto', 'matsuo']
author_list += ["maxwell", "grant", "bronston", "mayes", "treena", 'gillespie', "finney", "thomas", "shen"]
author_list += ["david", "mcconville", "john", "arnold", "alison", "smith", "leanne", "mccormick", "ross"]
author_list += ["donohue", "martin", "mccracken", "denise", "currie", "jeanette", "harrison", "aoife", "mcdermott", "edel"]
author_list += ["conway", "kenneth", "cafferkey", "janine", "bosak", "flood", "paula", "mcdonald", "sara", "charlesworth"]
author_list += ["tina", "graham", "anthony", "mcdonnell", "gunnigle", "jonathan", "lavelle", "ryan", "lamare", "ciarán"]
author_list += ["mcfadden", "marian", 'crowley', "henry", "mckinney", 'emma', "louise", "swartz", "leslie", "yvonne"]
author_list += ["mcnulty", "chris", "brewster", "yvonne", "mcnulty", "kate", "hutchings", "yvonne", "mcnulty"]
author_list += ["mcphail", "cristina", "inversi", "tony", "dundon", "nechanska", "ruth", "mcphail", "yvonne", "mcnulty", "kate", "hutchings"]
author_list += ["medina", 'garrido', "aurelio", "biedma", 'ferrer', "maría", "ramos", 'rodríguez']
author_list += ['antonio', "rafael", "jeroen", "meijerink", "anna", "nehles", "leede", "kamel", "mellahi"]
author_list += ["jedrzej", 'george', "frynas", "collings", "meng", "yang", "jianbing", "jizhu", "juzhe", "klaus", "meyer"]
author_list += ["katherine", "maryline", "meyer", "marc", "ohana", "florence", "stinglhamber", "meyers", 'maria']
author_list += ["christina", 'woerkom', "marianne", "paauwe", "jaap", "dries", "nicky", "timo", "meynhardt", "steven"]
author_list += ["brieger", "carolin", "hermann", "qing", "miao", "alexander", "newman", "huang", "miao"]
author_list += ["rentao", "bozionelos", "nikos", "zhou", "wenxia", "newman", "alexander", "marco", "michelotti", "andrea"]
author_list += ["vocino", "peter", "gahan", "julia", "roloff", "carla", "millar", "stephen", "chen", "waller"]
author_list += ["maura", "mills", "russell", "matthews", "jaime", "henning", "vivian", "atieh", "mirfakhar", "jordi"]
author_list += ["trullen", "mireia", "valverde", "sophie", "mitra", "douglas", "kruse", 'audra', 'mockaitis', 'lena']
author_list += ['zanderhelen', 'cieri', "audra", "mockaitis", "lena", "zander", "helen", "cieri", "miriam", "moeller"]
author_list += ["timothy", "kiessling", 'eric', 'molleman', "francesco", "montani", "jean", 'sébastien', "boudrias", "marilyne"]
author_list += ["pigeon", 'fiona', 'moore', 'moore', 'fiona', "david", "morand", "zhang", "linda", "mory", "bernd"]
author_list += ["wirtz", "vincent", "göttel", "klaus", "moser", "verena", "kemter", "kerstin", "wachsmann", "nora"]
author_list += ["köver", "roman", "soucek", "ahmed", 'mohammed', 'sayed', "mostafa", 'seymour', "williams", 'paula', 'mowbray']
author_list += ["noel", "muniz", "josé", 'ariza', "montes", 'leal', "rodríguez", "murphy", "caroline", "cross"]
author_list += ["christine", "peter", "murray", "faiza", "liisa", "mäkelä", "ulla", "kinnunen", 'paolo']
author_list += ['neirotti', "nekhili", "mehdi", "boukadhaba", "amal", "nagati", "haithem", "chtioui", "tawhid", "philippe"]
author_list += ["nelissen", "hülsheger", "gemma", "ruitenbeek", "fred", "zijlstra", "alexander", "newman", "christina"]
author_list += ["butler", "alex", "newman", "susan", "mayson", "teicher", "rowena", "barrett", "alexander", "newman"]
author_list += ["ingrid", "nielsen", "qing", "miao", "alexander", "qing", "miao", "hofman", "cherrie", 'jiuhua']
author_list += ["neesham", "graham", "manville", "herman", "newman", "alexander", "mayson", "susan", "teicher", "julian"]
author_list += ["barrett", "rowena", "cameron", "newton", "alicia", "mazur", "cameron", "newton", "adelle", "bish"]
author_list += ["renata", "anibaldi", "vicky", "browning", "drew", "thomas", "phuong", "nguyen", "jörg", "felfe"]
author_list += ["insa", "fooken", "anna", 'maija', "nisula", "aino", "kianto", "eimear", 'marie', "nolan", "michael"]
author_list += ["morley", "collins", "ntim", "lindop", "thomas", "hussein", "abdou", "kwaku", "opong", "helge", "nuhn"]
author_list += ["sven", "heidenreich", "andreas", "wald", "nyberg", "gregory", "reilly", "spenser", "essman", "jenna"]
author_list += ["rodrigues", "oehmichen", "mariano", "heyden", "dimitrios", "georgakakis", "henk", "volberda", "chidiebere"]
author_list += ["ogbonnaya", "danat", "valizade", "paul", "ogunyomi", "nealia", "bruning", "andrea", 'ollo', "lópez", "salomé", 'goñi']
author_list += ["legaz", "sophie", "beeck", "wynen", "annie", "hondeghem", "oppel", "vera"]
author_list += ["winter", "jonas", "schreyögg", "viktoria", "oppenauer", "karina", "voorde", 'ortiz', "guinea"]
author_list += ["jane", "webster", 'lilian', 'otaye', 'ebede', "otterbach", "steffen", "charlwood", "andy", "king", "wooden"]
author_list += ["mark", "zhanying", "wang", "tingting", "chen", "ouyang", "xiangmin", 'helen', "zhengtang", "zhang", 'alison', 'owens'] 
author_list += ["mustafa", 'bilgehan', "ozturk", "tatli", "elaine"]
author_list += ["brien", "carol", "linehan", "wayne','donohue", "nuttaneeya", "torugsa", 'wayne', "’donohue"]
author_list += ["hutchings", "samantha", "montes", "wayne", "hansen", 'pascal', 'paille', 'francois', 'grima', 'marie', 'dufour', "varina"] 
author_list += ["paisley", "mark", "tayar", "long", "emmanouil"]
author_list += ["papavasileiou", "sean", "lyons", 'praveen', "parboteeah", 'titilayo', "seriki", "martin", "hoegl"]
author_list += ['rhokeun', 'park', "rhokeun", "park", 'jung', "jang", "jisung", "park", "seongsu", "joon"]
author_list += ['hyung', "park", "masakatsu", "joon", 'hyung', "park", "alexander", "lili", "chendong", "angus"]
author_list += ["hooke", "sohee", "park", "sehee", "yujin", "owwon", "park", "johngseok"]
author_list += ["woonki", "hong", "polly", "parker", "richard", "cotton", "yates", "janeen", "baxter", "arend", "parry"]
author_list += ["emma", "dickmann", "michael", "cooke", 'fang', "lepak", "dave", "berber", "eisinga", "hans"]
author_list += ["doorewaard", "susana", "pasamar", "ramón", "valle", "judy", "pate", "hugh", "scullion", "samantha"]
author_list += ['paustian', "underdahl", "lisa", 'slattery', "walker", 'matteo', 'pedrini', "peltokorpi", "vesa"]
author_list += ["ramaswami", "aarti", "hasuli", 'kumarika', "perera", 'teng', "chew", "ingrid", "nielsen", "hilla", "peretz", "ariel"]
author_list += ["levi", "yitzhak", "fried", "pascale", "peters", "ligthart", "anne", "bardoel", "erik"]
author_list += ["poutsma", "mark", "peterson", "iterson", "agnieszka", "piasna", "janna", "rose", "jill", "rubery"]
author_list += ["brendan", "burchell", "anthony", "rafferty", "shaun", "pichler", "gerard", "beenen", "stephen"]
author_list += ["wood", "placide", 'poba', "nzaou", "sylvestre", "uwizeyemungu", "carlene", "clarke", "richard", "posthuma"]
author_list += ["jase", "ramsey", "gabriela", "flores", "carl", "maertz", "rawia", "ahmed", "taman", "powell"]
author_list += ["véronique", "ambrosini", 'alfred', 'presbitero', "alfred", "presbitero", "lemuel", "toledano", "alfred"]
author_list += ["presbitero", "banjo", "roxas", "doren", "chadee", "presbitero", "alfred", "huong", "jiang", "zhou"]
author_list += ["zheng", "connie", "robin", "price", "janis", "bailey", "amanda", "pyman", "isabel", 'prieto']
author_list += ["pastor", "victor", 'martin', "perez", "stephen", "procter", "radnor", "prouska"]
author_list += ["alexandros", "psychogios", "alexandros", "psychogios", "fotis", "missopoulos", "andrej", "kohont"]
author_list += ["elizabeta", "vatchkova", "agnes", "slavic", "markus", "pudelko", "reiche", "carr", "valeria", "pulignano", "nadja"] 
author_list += ["doerflinger", "katarina", "putnik", "inge", "houkes", "nicole", "frans", "nijhuis", "ijmert"]
author_list += ["kant", "francisco", 'pérez', "conesa", "romeo", "montserrat", 'yepes', "baldó", "jing", "zhuo", 'rachel']
author_list += ["haiwan", "wang", "chuying", "baihe", "song", "rujie", "onne", "janssen", "samina"]
author_list += ["quratulain", "abdul", 'karim', "crawshaw", "ghulam", "arain", "imran", "hameed", "tanja", "rabl"]
author_list += ["marı´", 'carmen', "triana", "girts", "racko", "eivor", "oborn", "michael", "barrett", "sumita"]
author_list += ["raghuram", 'ping', "elaine", "farndale", "stan", "gully", "morley", 'andrés', 'raineri', "boumediene"]
author_list += ["ramdani", "kamel", "cherif", "guermat", "rabah", "kechad", "jacobo", "ramirez", "sergio", "madero"]
author_list += ["carlos", "muñiz", "rekha", "zaheer", "hemant", "merchant", "tarba", "claudio", "ravasi", "xavier"]
author_list += ["salamin", "eric", "davoine", "katherine", "ravenswood", "raymond", "markey", "julie", "rayner"]
author_list += ["daniel", "espinoza", "bruce", "rayton", "zeynep", "yalabik", "douglas", "renwick", "charbel", "jabbour"]
author_list += ['muller', "camen", "redman", 'wood', "sarpong", "david", "maclean", "mairi", "sarvaiya", "harsha"]
author_list += ["arrowsmith", "eweje", "gabriel", "saundry", "bennett", "wibberley", 'adem', "andrew", "schenkel", "robin", "teigland"]
author_list += ["schmid", "andreas", "sender", "anna", "joseph", "schmidt", "chelsea"]
author_list += ["willness", "jones", "joshua", "bourdage", "matthias", "schneid", "rodrigo", "isidor", "chengguang"]
author_list += ["rüdiger", "kabst", "martin", "schneider", "johanna", "flore", "schneider", 'martin', "iseke", "anja"]
author_list += ["pull", "kerstin", "dora", "scholarios", "hannah", "hesselgreaves", "pratt", "schopman"]
author_list += ["kalshoven", "boon", "bert", "schreurs", "hannes", "guenter", 'hetty', "emmerik", "notelaers"]
author_list += ["désirée", "schumacher", "lisa", "schur", "adrienne", "colella", "meera", "adya", "fabrizio"]
author_list += ["scrima", "lucrezia", "lorito", "giorgio", "falgares", "hannele", "seeck", "marjo", 'riitta', "diehl"]
author_list += ["seet", "jones", "janice", "acker", "jogulu"]
author_list += ['anil', 'swq', 'kdx', 'kdo', 'khb', 'neeraj', 'production','user', 'vikas','kumartechbooks', 'ambika','prasadtechbooks', 'ambika','otm', 'bamac6', 'ind', 'lll', 'pinnacle', 'fxz', 'blackwell','medical', 'ind']
author_list += ["hillie", "aaldering","lindred","greer", "gerben", "kleef", "carsten", "dreu","susanne"]
author_list += ["abele", "garold", "stasser", "christopher", "chartier", "hajo", "adam", "otilia", "obodaru", "adam"]
author_list += ["galinsky", 'hajo', 'adam', "gabrielle", "adams", "inesi", "madan", "pillutla", "ishani", "aggarwal"]
author_list += ["anita", 'williams', "woolley", "nidhi", "agrawal","dahee","adam", "duhachek", "emily"]
author_list += ["amanatullah","catherine","tinsley", "maureen", "ambrose", "marshall", "schminke", "scott"]
author_list += ["reynolds", "daniel","ames","elke","weber" "amit","lilach", "sagiv"]
author_list += ['eduardo','andrade', "eduardo','andrade', 'ariely", "frederik", 'anseel', 'filip', 'lievens', 'eveline']
author_list += ["schollaert", 'ayala', 'arad', "nicholas","aramovich","james","larson", "linda", "argote",]
author_list += ["erin", "fahrenkopf", 'argote','arkes','arkes', 'david', 'hirshleifer']
author_list += ['danling', 'jiang', "sonya", "rachel","arnett", "sidanius", "laurence", "ashworth", "peter"]
author_list += ["darke", "lindsay", "mcshane", "tiffany", "giuseppe", "attanasi", "aurora", 'garcía', "gallego"]
author_list += ["nikolaos", "georgantzís", "aldo", "montesano", "evelyn", "zhang", "brandy"]
author_list += ["aven","lily", "morse","alessandro", "iorio", 'derek','avery', "shahar", "ayal","jérémy"]
author_list += ["celse" "hochman", "markus", "baer","graham", "brown", "feng","grace", 'ching'] 
author_list += ["daniel", "balliet","lance", "ferris", "gary", "ballinger", "david", "lehman", "schoorman"]
author_list += ['baltes','baltes',"alixandra", "barasch", "emma", "levine", "maurice"]
author_list += ["schweitzer", "christopher", "barnes", "john", "schaubroeck", "megan", "huth", "sonia", "ghumman", "andrew"]
author_list += ["michael", "karpman", "sharon", "long", "yaniv", "hanoch", "thomas", "rice", "shankha", "basu",]
author_list += ["krishna", "savani", "bauman", "leigh", 'plunkett', "tost", "madeline","bazerman","ovul"]
author_list += ["sezer", "james","beck","aaron","schmidt", 'beersma',"bianca", 'beersma', 'john']
author_list += ['hollenbeck', 'donald', 'conlon', 'stephen', 'humphrey', 'henry', 'moon', 'daniel', "ilgen", "bianca", "beersma"]
author_list += ["astrid", "homan", "peter", "belmi","margaret", "neale", 'benbasat','justin','berg']
author_list += ["peter", "bergman", "jessica", 'lasky', "fink", "todd", "rogers", "jonathan", "berman", "amit"]
author_list += ["bhattacharjee", "deborah", "small", "zauberman", "john", "beshears", "james", "choi", "laibson", "brigitte"]
author_list += ["madrian", 'tilmann', 'betsch', 'susanne', 'haberstroh', 'andreas', 'ckner', 'rthomas', 'haar', 'klaus']
author_list += ['fiedler', "constant','beugré", 'syon','bhanot', 'rajesh', 'bhargave', "nazlı", "bhatia","brian"]
author_list += ["gunia", "emily","bianchi","joel", "brockner", "guido", 'biele', 'jörg', 'rieskamp']
author_list += ["czienskowski", 'baler', 'bilgin', "baler", "bilgin","lyle", "brenner", "mark", "bing", "kluemper"]
author_list += ['kristl', "davison", "shannon", "taylor", "milorad", "novicevic", "dipayan", "biswas","cornelia"]
author_list += ["pechmann",'bradford', 'bitterly', "steven", "blader", "batia", "wiesenfeld", "marion", "fortin", "sara"]
author_list += ['wheeler', "smith", 'sean', 'blair', "allen", "blay", "kathryn", "kadous", "kimberly", "sawers", 'blount',]
author_list += ["hayley", "blunden", "jennifer", "logg", "alison", 'wood', "brooks", "leslie", "john", "francesca"]
author_list += ["gino", 'boehne', 'fong', "wong", 'vanessa','bohns', 'boles']
author_list += ["bolino", "jaron", "harvey", "daniel", "bachrach", "bryan","bonner","alexander"]
author_list += ["bolinger", "simona', 'botti', 'christopher','hsee", "susanne", "braun","claudia", "peus",]
author_list += ["dieter", "frey", "lyle", "brenner","baler", "bilgin", "jeanne", "brett","leigh", "thompson"]
author_list += ['brief','brief',"sebastien", "brion","cameron", "anderson", 'brockner']
author_list += ['joel', 'brockner', 'brodt',"alison", 'wood', "brooks","maurice"]
author_list += ["schweitzer", 'alison', 'wood', 'brooks', "graham", "brown","markus", "baer", "patrick", "bruning"]
author_list += ["bradley", "alge", "hsin", 'chen', "roger", 'buehler', 'johanna', 'peetz', 'dale', "griffin", "roger", "buehler"]
author_list += ["dale", "griffin", "kent", "deslauriers", "katherine", "burson", "faro", "yuval", "rottenstreich", "marcus"]
author_list += ["butts", "devin", "lunt", "traci", "freling", "allison", "gabriel", "adrian","camilleri"]
author_list += ["newell", "kurt","carlson","abhijit", "guha", "kurt", "carlson", "robin", "tanner", "margaret"]
author_list += ["meloy", 'edward', "russo", "joel", "carnevale", "huang", "harms", "lisa", "cavanaugh", "gino", "gavan"]
author_list += ["fitzsimons", "brianna", 'barker', "caza", "larissa", "tiedens", "fiona", 'wook', "chang", "rosalind"]
author_list += ["chow", "anita", "woolley", "christopher","chartier","susanne", "abele", "jennifer", 'chatman']
author_list += ['alicia', 'boisnier', 'sandra', 'spataro', 'cameron', 'anderson', 'jennifer', "berdahl", "anne", 'sophie', "chaxe_listl"]
author_list += ["catherine", "wiggins", "jieru", 'xiao', 'ping', 'chen', "xiao", "chen","gary","latham", "xiao"]
author_list += ['ping', "chen" "morgan", "gilad", 'ruth', 'kanfer', 'richard', 'deshon', 'mathieu', 'steve', "kozlowski"]
author_list += ["chao", "chen", "friedman", "mcallister", "tingting", "fuli", "xiao", 'ping', "zhanying", "arik", "cheshin"]
author_list += ["anat", "rafaeli","nathan", 'arik', 'cheshin', "arik", "cheshin" "amit","gerben"]
author_list += ["kleef", "cheung", "emily", "song", "liqun", 'tsan', "chang","hsien", 'lier']
author_list += ["huang","sinhui", "chong", 'whee', "russell", "johnson", 'joanna', "jessica", 'siegel']
author_list += ["christian", "matthew", "pearsall", "erin", "long", "chua","michael"]
author_list += ["morris","shira", "jinseok", "chun", "joel", "brockner", "cremer", 'corinne', 'coen', 'cohen']
author_list += ['charash',"irina", "cojuharenco","natalia", "karelaia", "irina", "cojuharenco", "patient"]
author_list += ["bashshur", "jason", "colquitt", "kate", "zipay", "lynch", "ryan", "outlaw", "donald", "conlon", "tinsley"]
author_list += ["samuel", "birk", "stephen", "humphrey", "aleksander", "ellis", "benjamin","converse","patrick"]
author_list += ["dennis", 'copeland',"yann", "cornil","david","hardisty","yakov", "bart"]
author_list += ['coughlan',"william", 'cron', 'mary', 'gilly', 'graham', 'slocum", "matthew','cronin']
author_list += ['cleotilde', 'gonzalez', 'john','sterman", "craig',"crossley", "cynthia", "cryder", "george"]
author_list += ["loewenstein", "richard", "scheines", "jared", 'curhan', 'margaret', 'neale', 'ross', 'jesse', 'rosencranz']
author_list += ["engelmann", 'shawn','curleyglenn','browne', "serge",'motta', "veiga","daniel"]
author_list += ["turban", "dahling", "shaun", "wiley", "zachary", "fishman", "amber", "loihle", 'hengchen',"xianchi"]
author_list += ["zlatev", 'david','daniels', 'darr',"tina", "davidson","linn", "dyne"]
author_list += ["bilian", "mark','davis",'bruin',"david", 'cremer', 'eric', "dijk", "erik"]
author_list += ["kwaadsteniet", "sanne", "rijkhoff", "eric", "dijk", "frank", "karen", "jehn", "daan", "scheepers"]
author_list += ["michael', 'dekay', 'dalia', 'patiño', 'echeverri', 'paul', 'fischbeck", "michael", "dekay", "seth", "miller"]
author_list += ["schley", "breann", "erford", "rellie', 'derfler', 'rozin', 'madan', 'pillutla', 'stefan', 'thau", "sreedhari"]
author_list += ["desai","maryam", "kouchaki", 'sreedhari','desai', "pieter", "desmet","david"]
author_list += ["cremer","eric", "dijk", "dirks", "donald", "ferrin", "cecily", "cooper", "jeremy", "douthit"]
author_list += ["melanie", "millar", "roger", "white", 'ning',"ning", "budescu", "marjorie", "shelly", "omer", 'michelle']
author_list += ['duguid', "james', 'dulebohn', 'issidoros', 'sarinopoulos', 'robert', 'davison', 'gerry', 'mcnamara", "jennifer"]
author_list += ["dunn", "nicole", "ruedy", "sunasir", "dutta","hayagreeva", 'charlotte', 'mayne', "jane"]
author_list += ["ebert", "mahdi", "ebrahimi", "maryam", "kouchaki", "vanessa", "patrick", 'none', 'emir', 'efendić', "daniel"]
author_list += ["effron","dale","miller", "daniel", "effron", "brian", "lucas", "kieran", "’connor", "casey"]
author_list += ["eggleston", "timothy", "wilson", "minha", "gilbert", 'kyle','emich', "kyle","emich","lynne"]
author_list += ["vincent", "kimmo", "eriksson", "pontus", "strimling", "julie", "coultas", 'arvid', 'erlandsson', "lauren"]
author_list += ['eskreis', "winkler","ayelet", "fishbach", "nathanael", "fast", "niro", "sivanathan", "mayer", "sean"]
author_list += ["fath","aaron" "alberto", "feduzi","jochen", "runde", "sanne", "feenstra", "jordan"]
author_list += ["frank", "walter", "janka", "stoker", "ryan", "fehr","michele","gelfand", "ryan", "fehr", "jack"]
author_list += ['ting', "chiang", "welsh", "baer", "manuel", "vaulont", "philip", "fernbach", "york", "hagmayer", "steven"]
author_list += ["sloman", "donald', 'ferrin', 'michelle', 'bligh', 'jeffrey', 'kohles", 'lance', 'ferris', 'douglas']
author_list += ['brown', 'daniel', "heller", "florian", 'fessel', 'epstude', 'neal',"roese", 'fiedler',"klaus"]
author_list += ["fiedler", "franz", "wöllert", "benjamin", "tauber", "philipp", "hess", "susann", "andreas", "glöckner"]
author_list += ["nicklisch", "stephan", "dickert", "fischer", "kastenmüller", "tobias", "greitemeyer", "julia", "dieter"]
author_list += ["frey", "ayelet", "fishbach","jinhee", "choi", 'colin','fisher', 'forgas',"alison"]
author_list += ['fragale', 'benson', 'rosen', 'carol', 'iryna', "merideth", "douglas", "frank", "klaus", "wertenbroch"]
author_list += ["william", "maddux", 'fraser', "mackenzie","sung", "johnson", "traci", "vincent"]
author_list += ["henard", "jane", "fung","wing", 'tung', "deshani","ganegoda","robert", "folger"]
author_list += ['ganzach',"stephen", "garcia", "hyunjin", "song", "abraham", "tesser", "varun", "gauri"]
author_list += ["julian", "jamison", "nina", "mazar", "owen", "ozier", 'phillip',"jennifer","george","erik"]
author_list += ["dane", "robert','giambatista', 'anita','bhappu", 'gibson',"kerry", 'roberts']
author_list += ["gibson", "dana", "harari", 'carson', "marr", 'gilbert',"francesca", "gino", "francesca"]
author_list += ["gino","adam","galinsky", "francesca", "gino","joshua","margolis", "francesca", 'gino']
author_list += ['lamar', "pierce", "francesca", 'gino', 'shang', 'rachel', "croson", "francesca", 'gino', 'linda', 'argote', 'ella']
author_list += ['miron', 'spektor', 'gergana', "todorova", "francesca", 'gino', 'lisa',"bazerman", "francesca"]
author_list += ["zachariah", "sharek", "moore", "schweitzer", "mead", "ariely", "francesca", "gino","ovul", "sezer"]
author_list += ["laura", "huang", "eric", "gladstone","kathleen","’connor", "jack",'goncalo', 'michelle']
author_list += ["duguid", "jack","goncalo","michelle","duguid", "jack", "goncalo", "evan", "polman"]
author_list += ["christina", "maslach", 'helena','gonzã¡', 'gã³mez', 'gonza´', 'vallejo',"claudia", 'gonzã¡']
author_list += ["vallejo", "jason", "harman", "etienne", "mullet", "maria", 'muñoz', "sastre", "jodi", "goodman", "robert"]
author_list += ["wood", "zheng", "chen", "james", "grand", "juliya", "golubovich", 'marie', "ryan", "neal", "schmitt"]
author_list += ["adam","grant","david","hofmann", "adam','grant', 'sabine', 'sonnentag", "adam", "grant"]
author_list += ["samir", "nurmohamed", "susan", "ashford", "dekas", "lindred", "heather", "caruso", "jehn"] 
author_list += ['gruenfeld',"cristiano","guarana","christopher","barnes", "brian","gunia"]
author_list += ["emma","levine", 'guyse',"philip", "gylfe","henrika", "franck","eero"]
author_list += ["vaara", "andrew","hafenbrack","kathleen","vohs", "hafenbrack", "lindsey", "cameron"]
author_list += ["gretchen", "spreitzer", "chen", "zhang", "laura", "noval", "samah", "shaffakat", 'lydia', 'paine', 'hagtvedt']
author_list += ["jeffrey', 'hales", "rebecca', 'hamilton', 'stefano', 'puntoni', 'nader', 'tavassoli", "hamilton", "kathleen"]
author_list += ["vohs", "anne", 'laure', "sellier", "meyvis", "sean", "hannah", "bruce", "avolio", "fred", "walumbwa"]
author_list += ["adrian", "chan", "jennings", "dustin", "bluhm", 'chunyan', "peng", "schaubroeck", 'uriel', 'haran', 'andrew']
author_list += ['hardin', 'hardy', 'anthony', "hughes", "xiaoqian", "wang", "schuelke", 'harinck',"einav"]
author_list += ["hart","maurice","schweitzer", 'harvey',"neil", "hauenstein", "kevin", "bradley"]
author_list += ["patrick", 'gavan', "’shea", "yashna", "shah", "douglas", "magill", "madeline", "heilman", "manzi", "suzette"]
author_list += ["caleo", 'henry','sandy', "hershcovis", "lukas", "neville", "tara", "reich", "christie"]
author_list += ["lilia", "cortina", 'valerie', "shan", "hershfield", "taya", "cohen", "thompson", "ivona", "hideg"]
author_list += ['lance', "ferris", "ivona", "hideg","anne","wilson", 'tory', "higgins","james"]
author_list += ["cornwell", 'angus', "hildreth", "bazerman", 'sharon', 'hill', 'kathryn', 'bartol', 'tesluk', 'gosia']
author_list += ["langa", 'hinds',"violet","dejun", 'tony', "kong", "hollenbeck", "ellis"]
author_list += ["adela", "garza", "ilgen", "peter","zhixing", "xiao", 'horvath',"lawrence"]
author_list += ["houston", "alicia", "grandey", "katina", "sawyer", "hsee", "luxi", "shen", "shirley", "jingqiu", "zhang"]
author_list += ["christopher","hsee","yang", "yang","xilin", "kaifeng", "jiang", "shenjiang", "honghui"]
author_list += ["junqi", "yanliu", "huang",'wesley', "hutchinson", ", 'huang','keith', 'murnighan",'huang']
author_list += ["dong", "huang","liyin","ying", "zhang", 'huber',"michaela", "huber"]
author_list += ["leaf", "boven", 'peter', "mcgraw", 'johnson', "graham", "hilde", "huizenga", "ruud", "wetzels", "ravenzwaaij"]
author_list += ["wagenmakers", "federico", "aime", "lily", "cushenbery", "aaron", "hill", "joshua", "fairchild", 'daniel']
author_list += ['ilgen', "remus", "ilies", "keeney", "brent", "scott", "lynn", 'imai', 'michele',"gelfand", "inesi"]
author_list += ["chester", "insko", "wildschut", "cohen", 'julie','irwinjonathan', 'baron', "jachimowicz", "shira"]
author_list += ["agasi", "stéphane", "côté", 'jack', "walker", "hubert", "feild", "giles", "jeremy", "bernerth", "short"]
author_list += ["karen','jehn', 'katerina', 'bezrukova", "lars', 'olof', 'johansson', 'henrik', 'svedsäter", "leslie"]
author_list += ["rick", "alessandro", "acquisti", "joachim", "vosgerau", "martha", "jeong", "huang", 'johnson']
author_list += ['scott', "derue", "jundt", "stefanie", "sitzmann", 'thuy', "nguyen", "russell", "danielle", "king"]
author_list += ["scott", 'jackson', "wang", "eric","jones","janice","kelly", "dong", "mingpeng", "dong"]
author_list += ["qiongjing", "marie", "juanchich", "miroslav", "sirota", "butler", 'judge',"timothy"]
author_list += ["judge","remus", "ilies","zhen", "zhang", "jung","sujin",'juvina', "ariel"]
author_list += ["kalil", "sebastian", "gallegos", "ruth", "kanfer","gilad", "chen", "dana", "kanze", "mark", "conley"]
author_list += ['tory', "higgins", "gokhan", "karagonlar","david","kuhlman", 'natalia', 'karelaia', "yoshihisa"]
author_list += ["kashima", "simon", "laham", "bianca", "levis", "darlene", "wong", "melissa", "wheeler", "edgar"]
author_list += ["kausel","jerel","slaughter", "edgar", "kausel", "satoris", "culbertson", "pedro", "leiva"]
author_list += ["jerel", "slaughter", "alexander", "jackson", "hector", "madrid", "eric", "kearney", "meir", "shemla"]
author_list += ["knippenberg", "florian", "scholz", 'steffen', 'keck', 'kelly',"jessica","kennedy",]
author_list += ["cameron", "anderson", "jessica","kennedy","maurice","schweitzer", "kennedy", "cameron"]
author_list += ["anderson", "kray", "gillian", "yeun", 'joon',"chen", "zhong", "cooper", "kurt", "ferrin", "linn"]
author_list += ["dyne", "dishan", "kamdar", "johnson", "hakkyun","kyoungmi","kiwan", "park", "kinias"]
author_list += ["heejung", "andrew", "jina", 'nadav', 'klein', 'klein',"klein", "knight", "jonathan"]
author_list += ["ziegert", "beng", 'chong', "saltz", "jesse", "kluver", "rebecca", "frazier", "haidt", 'knez']
author_list += ['tehila', 'kogut', "michail", "kokkoris", "baumeister", "ulrich", "kühnen", "lukas", "koning", "wolfgang"]
author_list += ["steinel", "ilja", "beest", "koopman", "fadel", "matta", "conlon", "shirli', 'kopelman", 'maryam', 'kouchaki']
author_list += ["maryam", "kristin", 'smith', "crowe", "arthur", "brief", "carlos", "sousa", 'kozlowski']
author_list += ["thomas", "kramer","lauren","block", "thomas", "kramer", "ozge", 'yucel', "aybat", "loraine"]
author_list += ["gesk", "michal", "maimaran", "itamar", "simonson", "florian", "kraus", "ahearne", "wieseke", 'kray']
author_list += ['kray',"laura", "alex", "zant", 'krishnamurthy',"justin", 'kruger', 'kenneth']
author_list += ["savitsky", "tamar", "kugler","gary", "bornstein", 'daniella', 'kupor', "letty","kwan"]
author_list += ["suhui", "chiu", 'kwong','¨hberger',"chak", "elizabeth"]
author_list += ["karam", "hollenbeck", 'klodiana', 'lanaj', 'lance','lance', "ferris", "shereen", "fatimah"]
author_list += ["ming", "lindie", "liang", "huiwen", "lian", "brown", "bart", "langhe", "stijn", "osselaer", "berend"]
author_list += ["wierenga", "andrea", "larney","amanda", "rotella" "barclay", 'kristin', 'laurin', 'frank']
author_list += ['schmidt', 'james', 'harter', 'kristy', "lauver",'david', 'lebel', "robyn",'leboeuf', 'eldar', 'shafir']
author_list += ['julia', 'belyavsky', "bayuk", "alice","daniel","ames", 'jooa', 'julia','fiona']
author_list += ['leelarissa','tiedens', "sunyoung", "marko", "pitesa", "pillutla", "stefan", "thau", 'whee',]
author_list += ["choi","seongsu", "alice", "loschelder", "martin", "schweinsberg", "malia", "mason", "tomã¡"]
author_list += ["lejarraga","cleotilde", "gonzalez", "geoffrey", "leonardelli", "geordie", "mcruer", "victoria", 'husted']
author_list += ["medvec", 'lepine',"sophie", "leroy", "sophie", "leroy","aaron","schmidt", "lisa"]
author_list += ['leslie', 'michele',"gelfand", 'levin','levin',"emma"]
author_list += ["levine","maurice","schweitzer", "emma","levine","kristina","wald", 'levine',"meng","gretchen","chapman","xilin","christopher","hsee"]
author_list += ["evans", "gilliland", "edgar", "jordan", "stein", "shaobo", 'kevin', "michail", "krishna", "savani", "huiwen",]
author_list += ["alicea", "lieberman","kristen","duke" "amir", "michael",'liersch', 'craig']
author_list += ["mckenzie", "katrina", "remus", "helen", "pluut", 'ying', "fern", "healy","deborah","small", 'lind',"friedman","ying"] 
author_list += ["hong", "peggy", "troy", "campbell", "fitzsimons"]
author_list += ["gráinne", "shalley", "sejin", "keem", "jing", "zhou", 'xiao', "ying", "hong", 'pong', "yongfang", "jiangli"]
author_list += ["jiao", "stefano", "livi", "arie", "kruglanski", "antonio", "pierro", "lucia", "mannetti", "kenny", "minson"]
author_list += ["robert", "lord", "paola", "gatti", "susanna", "chui", 'robert','lount','lount', "nathan"]
author_list += ["pettit", "sarah", "doyle", "jingyi","xiaofei", "jackson", "modupe", "akinola", "mason"]
author_list += ["shenghua", "luan","jochen", 'luce',"matthew", "lupoli", 'eric', "greenberg"]
author_list += ["nicholas','lurie', 'jayashankar','swaminathan", "brent","lyons","brent","scott"]
author_list += ['anyi',"anyi", "yang","krishna", "savani", 'macchi','robert','maccoun']
author_list += ["boris", "maciejovsky","david","budescu", "carl",'maertz', 'ahmad', 'hassan', 'peter']
author_list += ["magnusson", "vincent","amnon", "rapoport", "vincent","amnon", "rapoport","darryl"]
author_list += ["seale", "vincent", "rami", "zwick", "akshay", "jake", "pattaratanakun", "deepak', 'malhotra", "selin"]
author_list += ["malkoc", "zauberman", "bettman", 'mandel',"shannon", "marlow", "lacerenza", "jensine"]
author_list += ["paoletti", 'shawn', "burke", "eduardo", "salas", "marr", "thau", "karl", "aquino", "laurie", "barclay"]
author_list += ["ashley","martin","katherine","phillips", "martin", "bidhan", 'lalit', "parmar", "elena"]
author_list += ["martinescu", "onne", "janssen", "bernard", "nijstad", "malia",'mason', 'rebecca', 'dyer', 'michael']
author_list += ["norton", "pragya", "mathur", "shailendra", 'pratap', "jain", "meng", "hsieh", "charles", "lindsey"]
author_list += ["durairaj", "maheswaran", 'matsumoto',"david", 'mayer', 'maribeth', 'kuenzi', 'rebecca', 'greenbaum']
author_list += ['bardes', 'rommel', 'bombie', "salvador", "david", "kristina", "workman", "marius", "dijke", "linda", 'klebe']
author_list += ["treviño", "debra", "shapiro", "schminke", 'ketti', 'mazzocco', "matthew", 'mccarter', 'kevin', 'rockmann']
author_list += ['gregory', "northcraft", "mccarter", "jürgen", "scheffran", "julie", "mccarthy", "talya", "bauer", "truxillo"]
author_list += ["campion", "chad", "iddekinge", "campion", "mcelroy", "summers", "kelly", "todorov", "howard", "kunreuther"]
author_list += ['mckenzie',"craig", 'mckenzie', 'michael','liersch', 'ilan', "yaniv", "geeta"]
author_list += ['menon', 'ellie','kyung', 'nidhi', "agrawal", "tanya", "menon", "jeanne", "chiu", "hong", "christoph"]
author_list += ["merkle","martin", "weber", 'mesmer', "magnus", "dechurch", "miliani", 'jimenez', "rodriguez", "wildman"]
author_list += ["marissa", "shuffler", "kerry', 'milch', 'elke', 'weber', 'kirstin', 'appelt', 'michel', 'handgraaf', 'krantz"]
author_list += ["edward","miles","elizabeth","clenney", 'katherine','milkman', "milkman", "mary", 'carol']
author_list += ["mazza", "lisa", "chia", 'jung', "tsay", "kobe", "millet","lien", "lamey","bram", "bergh"]
author_list += ["andrew','miner', 'theresa','glomb", "julia", "vanepps", "ella", 'miron', "spektor","gerard"]
author_list += ["beenen", "ella", "spektor", "argote", "dorothee", "mischkowski", "lewisch", "himanshu", "mishra", "arul"]
author_list += ["rixom", "promothesh", "chatterjee", 'alexandra','mislin', "daniel", "mochon","shane", "frederick"]
author_list += ["susan", "mohammed","david","harrison", 'mohammed',"welmer", "molenmaker", "erik"]
author_list += ["molinsky", "margolis", 'marlon', 'mooijman', "celia", "moore","tenbrunsel", "sarah", "moore"]
author_list += ["neal", "baba", "shiv",'moore', 'moreland','morgan',"morris"]
author_list += ['jennifer','mueller', "laetitia","mulder","karl", "aquino", "laetitia", "mulder", "floor"]
author_list += ["rink", 'keith', "murnighan","long", "wang", 'brian','murtha', 'mussweiler']
author_list += ['nahrgang', 'frederick', 'morgeson', 'remus', "ilies", "nahrgang", "matthias", "spitzmuller", "jundt", "inbal"]
author_list += ['nahum', "shani","peter","bamberger", "nathaniel", "nakashima", "daniels", "laurin", "jayanth"]
author_list += ["narayanan","kenneth", "kinias", 'nederveen', "pieterse", "wendy", "ginkel", 'nelson','daniel','newark', "newark", "bohns"] 
author_list += ["francis", "flynn",'nglinn', 'dyne']
author_list += ["linn", "dyne","soon", "nicklin", "greenbaum", "laurel", "mcnall", "folger", "williams", "nicos"]
author_list += ['nicolaou', 'scott', 'shane', 'lynn', 'cherkas',"spector", "norton", "carney", 'laura','noval']
author_list += ["günter", "stahl", 'kathleen','’connorjosh','arnold', "burak", "diefendorff", "bashshur", "gary"]
author_list += ["greguras", 'ofir',"greg","oldham","yitzhak", "fried", 'madeline','ordo','overbeck', 'cassandra', "govan",'hweta','joydeep','srivastava', 'nevena']
author_list += ['koukova", "neeru', 'paharia', 'karim', 'kassam', 'joshua', 'greene', "bazerman", "neeru", "paharia", "rohit"]
author_list += ["deshpandé", 'mauricio', 'palmeira', "karthik", "panchanathan", "willem", "frankenhuis", "joan", "silk"]
author_list += ["guihyun", "park","richard","deshon", 'youn', "park","seongsu", "sung"]
author_list += ["shefali","patil",'david', "lebel", 'paulus',"craig", "pearce"]
author_list += ["wassenaar", "yair", "berson", "rivka", 'tuval', "mashiach", 'pearsall', 'aleksander', 'ellis', 'jordan', "stein"]
author_list += ['peng', 'pettibone','nathan', "doyle", "michel', 'tuan', 'pham', 'tamar', 'avnet"]
author_list += ["michel", 'tuan', "pham", 'faraji', "olivier", "toubia", "leonard", 'jean','phillips', 'david']
author_list += ["piercey", "nashita", "pillay", "guihyun", "park", 'kang', "sujin", "robin", "pinkley", "sawyer", "sleesman"]
author_list += ["vandewalle", "maribeth", "kuenzi", "marko", "timothy", "pleskac", "stephanie", "merritt", "schmitt"]
author_list += ["frederick", "oswald", "helen", "petru", "curşeu", "yukun", 'evan', 'polman', "evan", "polman"]
author_list += ['edward', "russo", "christine','porath', 'amir', 'erez", "posavac", "kardes", 'joško', "brakus", 'kenneth']
author_list += ['price', 'thomas', 'hall', 'kees', 'hunton', 'lovett', 'mark', 'tippett', "devon", "proudfoot","aaron"]
author_list += ["heather", "mann", "christina", "rader", "soll", "larrick", "joseph','radzevick','moore"]
author_list += ["joseph","radzevick","moore", "tage","daniel", "diermeier", 'rapoport', 'services', 'rettinger','anine', 'riege'] 
author_list += ["sacramento", "doris", "west", "sunita"]
author_list += ["moore","robert","maccoun", 'daniel','schwartz', 'elizabeth','keenan', 'alex','imas']
author_list += ['ayelet','gneezy', 'robert', 'sneddonr', 'duncan', 'luce', "antonios", "stamatogiannakis", "amitava"]
author_list += ["chattopadhyay", "dipankar", "chakravarti", 'stone','unknown', 'gabriel', 'szulanski']
author_list += ['francesca', 'gino', "christian", "¶ster", "ajay", "mehra", "knippenberg", 'chia', 'jung', 'tsay', 'michel']
author_list += ['tuan', 'pham','meyvisrongrong', 'zhou', 'venkatesh','weiss', 'cohen', 'konstantinidis']
author_list += ['speekenbrink', 'mharvey','windschitl','yaniv','zeelenberg']
author_list += ['ines', 'meyer','stuart','carr','lori', 'foster', 'joseph','allen','nale', 'lehmann']
author_list += ['willenbrock', 'steven', 'rogelberg', 'david', 'antons', 'mathieu', 'declerck', 'kathleen', 'diener', 'iring', 'koch']
author_list += ['frank', 'piller', 'kara', 'arnold', 'catherine', 'connelly', 'gellatly', 'megan', 'walsh', 'michael', 'withey']
author_list += ['susan', 'ashford', 'wellman', 'mary', 'sully', 'luque', 'katleen',  'stobbeleir', 'melody', 'wollan', 'blake']
author_list += ['ashforth', 'glen', 'kreiner', 'mark', 'clark', 'fugate', 'neal','ashkanasy', 'uqbweave', "neal", "ashkanasy"]
author_list += ['william', "becker", 'david', "waldman", 'oluremi', "ayoko", 'karen', "jehn", "soroush", "aslani", 'jimena']
author_list += ['ramirez', "marin", 'jeanne', "brett", 'jingjing', 'zhaleh', 'semnani', "azad", "zhang", "tinsley", 'laurie']
author_list += ["weingart", 'wendi', "adair", 'prasad', 'balkundi','wang','rajiv', 'kishore', "boris", "baltes", 'kevin']
author_list += ["wynne", 'mgrdich', "sirabian", 'daniel', "krenn", 'annet', "lange", 'sarah', 'bankins', 'george', 'banks', 'sven']
author_list += ['kepes', 'mahendra', 'joshi', 'anson', 'seers', 'larissa','barberchristopher','budnick', 'william','beckerrussell'] 
author_list += ['cropanzano', 'thomas', 'becker', 'guclu', 'atinc', 'james', 'breaugh', 'carlson', 'jeffrey']
author_list += ['edwards', 'paul', 'spector', 'terry','beehr', "terry", "beehr", 'jennifer', "ragsdale", 'jonathan']
author_list += ["kochert", 'peter', 'belmijeffrey', 'pfeffer', 'andrew', 'bennett', 'bakker', 'field', 'carmen', 'binnewies','sabine']
author_list += ['sonnentag', 'mojza', 'michal', 'bironmarc', 'veldhoven', 'douglas','bonettthomas','wright', "anthony"] 
author_list += ["boyce", 'levi', "nieminen", "gillespie", 'marie', "ryan", "denison", 'kimberley']
author_list += ['breevaart', 'evangelia', 'demerouti', 'daantje', 'derks', 'nicola', 'breugst', 'rebecca', 'preller', 'holger']
author_list += ['patzelt', 'dean', 'shepherd', 'roman', 'briker','frank', 'walter','michael','cole', 'robert', 'buch','bård', 'kuvaas']
author_list += ['anders', 'dysvik', "claudia", "buengeler", 'astrid', "homan", "voelpel",'travis']
author_list += ['maynard', 'deanna', 'kennedy', 'christian', 'resick', 'anne', 'burmeister', 'yixuan', 'wang', 'junqi', 'yanghua']
author_list += ['vantilborgh','joeri', 'hofmans','timothy','judge', 'olivia', 'byza', 'sebastian', 'schuh']
author_list += ['stefan', 'dörr', 'matthias', 'spörrle', 'günter', 'maier', 'paul', 'harveymarie', 'dasborough', 'francesco']
author_list += ['cangiano', 'sharon', 'parker', 'gillian', 'dawn', 'merideth', 'thompson', 'wayne', 'crawford', 'michele', 'kacmar']
author_list += ['abraham', 'carmeli', 'stephen', 'brammer', 'emanuel', 'gomes', 'shlomo', 'tarba', 'mark','martinkojeremy','mackey']
author_list += ['filipa', 'castanheira', 'fabrice', 'cavarretta', 'laura', 'trinchera', 'dong', 'choi', 'sean', 'hannah']
author_list += ['lucia', 'ceja', 'heesun', 'chae','jisung', 'park','choi', 'melissa', 'chamberlin', 'newton', 'jeffery']
author_list += ['lepine','ling', 'chang', "jennifer", 'chatman', 'caldwell', 'charles', "reilly", 'bernadette', 'doerr']
author_list += ["caldwell",  "reilly", "doerr", "david", "charles", "bernadette", "doerr", 'tingting', 'chen','kwok']
author_list += ['leung','fuli','zhanying','ying', 'chen', 'zhen', 'xiong', 'lifeng', 'zhong', 'jooyeon', 'xiujuan']
author_list += ['zhang', 'zhiqiang', 'daniel', 'ming', 'chngjoyce', 'cong', 'ying', 'wang', 'jaee', 'chomichael','morris']
author_list += ['jaepil', 'andrea', 'kyongji', 'seongmin', 'jong', 'park', 'bora', 'kwon','chun','kyoungmin','john','sosik','hyeok']
author_list += ['chung','choi','jing','peter','colemankatharina','kugler']
author_list += ['brizzi', 'catherine', 'matej', 'černe', 'anders', 'dysvik', 'miha', 'škerlavaj', 'samantha','conroynina']
author_list += ['gupta', 'samantha', 'conroy', 'christine', 'henle', 'lynn', 'shore', 'samantha', 'stelman', 'neil', 'conway']
author_list += ['clinton', 'jane', 'sturges', 'budjanovcanin', 'jose','cortina','cristina', 'costa', 'ashley', 'fulmer']
author_list += ['neil', 'anderson','services', 'marcus', 'credépeter','harms', 'jeffrey','cucinamichael','mcdaniel', 'carla', 'gomes'] 
author_list += ['costa','zhou','aristides','ferreira', 'jason','dahlingmelissa','gutworth', 'marie','dasborough', 'marie']
author_list += ['dasboroughpaul', 'harvey', "emily", "david", 'derek', "avery"]
author_list += ["witt", 'patrick', "mckay", 'sarah', 'dawkins', 'tian', 'alexander', 'newman', 'angela', 'martin', 'kevin', 'dawson']
author_list += ['kimberly', "brien", 'terry', 'beehr','meulenaere','christophe', 'boone','tine', 'buyl', 'frank']
author_list += ['daan', 'scheepers', 'naomi', 'ellemers', 'sassenberg', 'annika', 'scholl', 'nathalie', 'delobbe', 'helena', 'cooper']
author_list += ['thomas', 'roxane', 'hong', 'deng','frank', 'walter','yanjun', 'guan', 'lindsay', 'dhanani', 'amanda', 'main']
author_list += ['andrew', 'pueschel', 'rebekah', 'dibble', 'mihaela', 'dimitrova', 'angela','dionisijulian', 'barling', 'direnzo']
author_list += ['marco', 'greenhaus', 'weer', 'christy','stanislav','dobrevjennifer', 'merluzzi', 'yuntao', 'kathryn']
author_list += ['bartol', 'chenwei', 'danyang', 'derks', 'chang', 'erin', 'eatough', 'laurenz', 'meier', 'ivana', 'igic', 'achim']
author_list += ['elfering', 'spector', 'norbert', 'semmer', 'ludvig', 'levasseur', 'abbie', 'shipp', 'yitzhak', 'fried', 'denise']
author_list += ['rousseau', 'philip', 'zimbardo', 'silke', 'astrid', 'eisenbeissdaan', 'knippenberg', "robert", "eisenberger"]
author_list += ['mindy', 'krischer', "shoss", 'gökhan', "karagonlar", 'gloria', 'gonzalez', "morales", 'robert', "wickham", 'louis']
author_list += ["buffardi", 'gabi', 'eissascott','lester', 'liat', 'eldoritzhak', 'harpaz', 'pervez', 'ghauri', 'john', 'child']
author_list += ['simon', 'collinson','parker', 'ellen','kyle','emichli','olga', 'epitropaki', 'ilias']
author_list += ['kapoutsis', 'ellen', 'gerald', 'ferris', 'konstantinos', 'drivas', 'anastasia', 'ntotsi', 'miriam', 'erez','andrew']
author_list += ['cynthia','jinyan', 'fanlei','steven','farmerlinn', 'dyne', 'kayla', 'follmer']
author_list += ['isaac', 'emmanuel', 'sabat', 'rose', 'siuta', "marion", "fortin", 'irina', "cojuharenco", "patient", 'hayley']
author_list += ["german", 'katherine', 'frear', 'paustian', 'underdahl', 'eric', 'heggestad', 'lisa', 'slattery', 'walker','ashley']
author_list += ['fulmercheri', 'ostroff', 'marylène', 'gagné', 'khee', 'seng', 'benjamin', 'katrina', 'hosszu', 'xiaohan']
author_list += ['urhahn','torsten', 'biemann','stephen','jaros', 'patrick', 'raymund', 'garcia', 'prashant', 'bordia']
author_list += ['lloyd', 'restubog', 'valerie', 'caines', 'michele', 'gelfand', 'severance', 'tiane', 'bayan', 'bruss', 'janetta']
author_list += ['abdel', 'hamid', 'latif', 'asmaa', 'ahmed', 'moghazy', 'sally', 'moustafa', 'ahmed', 'sonia', 'ghumman','marie']
author_list += ['ryan','park', 'elisabeth', 'gilbert','trevor', 'foulk','joyce', 'bono', 'paula', 'giordano', 'patient']
author_list += ['margarida', 'passos', 'francesco', 'sguera', 'barry', 'goldmanrussell', 'cropanzano', 'ryan','gottfredsonherman']
author_list += ['aguinis', 'laura', 'goverlinda', 'duxbury', "alicia", "grandey", 'deborah', "rupp", "brice", 'rebecca']
author_list += ['greenbaum', 'mawritz', 'julena', 'bonner', 'brian', 'webster', 'joseph', 'truit', 'gray', 'mawritz', 'martin']
author_list += ['gubler', 'yves', 'guillaume', 'jeremy', 'lilian', 'otaye', 'ebede', 'woods', 'west', "naina", "gupta", 'violet']
author_list += ["pollack", 'felipe','guzmanalvaro', 'espejo', 'dana','haggardhee', 'park', 'daniel', 'halgin']
author_list += ['borgatti', 'ajay', 'mehra', 'scott', 'soltis', 'angela', 'hall', 'dwight', 'frink', 'ronald', 'buckley', 'jing']
author_list += ['hanjuan', 'ling', "sean", "hannah", "sumanth", "lester", 'fabrice', "cavarretta", 'crystal','haroldbrian','holtz']
author_list += ['brad', 'harris', 'teresa', 'cardador', 'cole', 'mistry', 'bradley', 'kirkman', 'paul', 'harveymarie','dasborough']
author_list += ['jeffrey', 'haynie', 'mossholder', 'stanley', 'harris','ryan', 'fehr','rong', 'long','design', 'unit', 'london']
author_list += ['school', 'economics', "guido", "hertel", 'cornelia']
author_list += ["rauschenbach", 'markus', "thielgen", "krumm", 'ivona', 'hideggerben','kleef', 'higginmo', 'gerard','hodgkinsonj']
author_list += ['kevin', 'ford', 'joeri', 'hofmans', 'edina', 'dóci', 'omar', 'solinger', 'woohee', 'timothy', 'judge']
author_list += ['anthony', 'hood', 'bachrach', 'suzanne', 'zivnuska', 'elliot', 'bendoly', 'annekatrin', 'hoppe', 'toker', 'vivian']
author_list += ['schachler', 'ziegler', 'matt','howardrick','jacobs', 'ryan', 'shuwei', 'hsujames','stanworth']
author_list += ['xiaoxiao', 'huseth', 'kaplan', 'xiaoxiao', 'hujunqi','xiaoxiao', 'yujie', 'zhan', 'xiang', 'garden','huang','dina']
author_list += ['krasikova','peter','harms', "ronald", "humphrey", 'blake', "ashforth", "diefendorff"]
author_list += ['samuel', 'hunter', 'lily', 'cushenbery', 'jayne', 'seulki', 'jang', 'winny', 'shen', 'tammy', 'haiyan', 'zhang']
author_list += ["peter", "jennings", "mitchell", "hannah", 'jaclyn', 'jensen', 'rubin', 'yuan', 'jiang','susan','jackson','saba']
author_list += ['colakoglu', 'zhou', 'jiang', 'xiaowen', 'zhongmin', 'xuan', 'jiang', 'gary', 'johnsraghid']
author_list += ['hajj', 'andrew','johnsonkatherine','roberto', 'kristen', 'jones', 'eden', 'king', 'afra', 'ahmad', 'tracy']
author_list += ['mccausland', 'tiffani', 'chen', "dustin", "jundt", 'jason', "huang", 'jvanrooyen', 'gökhan', 'karagonlar']
author_list += ['eisenberger', 'justin', 'aselage', 'olli', 'pekka', 'kauppila', 'steffen', 'kecklinda', 'babcock', 'stacey','kessler']
author_list += ['kaitlin', 'kiburz', 'french', 'yeol', 'kyoung', 'yong','robert', 'eisenberger','kibok']
author_list += ['baik', 'sooyeol','youngah', 'park','qikun','linn', 'dyne','stephanie','scott','kingfred','bryant', 'danielle']
author_list += ['king','alexander', 'newman','fred', 'luthans']
author_list += ['howard', 'klein', 'anne‐kathrin', 'kleine', 'cort', 'rudolph', 'hannes', 'zacher', 'petra', 'klumb', 'manuel']
author_list += ['voelkle', 'siegler', 'michaela', 'knecht', 'bettina', 'wiese', 'alexandra', 'freund', 'kniffin', 'jubo', 'wansink']
author_list += ['schulze', 'caroline', 'knight','malcolm', 'patterson','jeremy', 'dawson', 'young', 'kojin', 'choi']
author_list += ['dohyoung','kyootai','kailash', 'joshi', "jaclyn", "koopmann", 'klodiana', "lanaj", 'joyce']
author_list += ["bono", 'kristie', "campana", 'jörg', 'korff','torsten', 'biemann','sven','voelpel', 'kristof']
author_list += ['brown', 'young', 'seong', 'degeest', 'seung', 'hong', 'mukta', 'kulkarni', "kunze","florian","menges",'jochen']
author_list += ['florian', 'kunze','stephan','boehmheike', 'bruch', 'lauren', 'kuykendall','lydia']
author_list += ['craig','louis','letty', 'kwanchi', 'chiu', 'jana', 'kühnel','ronald', 'bledow','nicolas']
author_list += ['feuerhahn', 'catherine','frank', 'walter','huang', 'lance', 'frazier', 'christina', 'tupper']
author_list += ['stav', 'fainshmidt', 'laurent', 'lapierre', 'elianne', 'steenbergen', 'maria', 'peeters', 'esther', 'kluwer']
author_list += ['lapierre', 'laurent', 'yanhong', 'kwan', 'kwong', 'direnzo', 'shao', 'ping', 'james', 'lavelle', 'christopher', 'rupp']
author_list += ['herda', 'randall', 'hargrove', 'meghan', 'thornton', 'lugo', 'gary', 'mcmahan', 'allan','sara', 'willis','tian', 'seth']
author_list += ['kaplan', 'carol', 'wong', 'kwok', 'leungjie', 'wang', 'author', 'alexander', 'lewisjonathan', 'clark','yuan', 'zhang']
author_list += ['xiao', 'ming', 'tian', 'zhaoli', 'song', 'richard', 'arvey', 'junchao', 'jason','tyler','burch','thomas','bertolt']
author_list += ['meyer','meir', 'shemla','jürgen']
author_list += ['wegge', 'schaubroeck', 'anita', 'keller', 'junchao', 'jason', 'barnes', 'cristiano', 'guaranalin', 'wang', 'jian']
author_list += ['liang','crystal',  'farh','liaohui', 'chun', "chenwei", "liao", 'sandy', "wayne"]
author_list += ["rousseau", "weipeng", "wang", "wang", 'dirk', 'lindebaumdeanna', 'geddes', 'dirk', 'lindebaumpeter','jordan']
author_list += ['jukka', 'lipponen','barbara', 'wisse','jolanda', 'jetten', "songqi", 'aleksandra', "luksyte", "zhou",'leigh']
author_list += ['anne','jian', 'dong', 'zhang', 'subrahmaniam', 'tangirala', 'cynthia', 'parker', 'natalia','lorinkovasara']
author_list += ['jansen', 'perry', 'fuli', 'kwok', 'leung', 'krishna', 'savani', 'morris', 'aleksandra']
author_list += ['luksytechristiane', 'spitzmueller', 'aleksandra', 'luksyte', 'kerrie', 'unsworth', 'avery','luria', "luria"]
author_list += ['yuval', "kalish", 'miriam', "weinstein", 'luria', 'allon', 'kahana', 'judith', 'goldenberg', 'yair', 'noam']
author_list += ['jeremy', 'mackey', 'charn', 'mcallister', 'brees', 'huang', 'jack', 'carson', 'leticia', 'maia', 'antônio', 'virgílio']
author_list += ['bittencourt', 'bastos', 'nathanaël', 'solinger', "muhammad", 'abdur', 'rahman', "malik", 'arif', "butt", "choi"]
author_list += ['muhammad', 'malik', 'nazir', 'butt', "charles", "manz", 'bruce', "skaggs", 'craig', "pearce", "wassenaar"]
author_list += ["mark", "martinko", "harvey", "mackey", 'suzanne','masterson', 'courtney','mastersonjenny','hoobler', 'john']
author_list += ['mathieu', "fadel", 'matta', 'tuğba', 'erol', 'korkmaz', 'russell', 'johnson', 'pinar']
author_list += ["b𝚤çaks𝚤z", "korkmaz", "johnson", 'ella', 'miron', 'spektor', 'susannah', 'paletz', 'chun', 'rebecca']
author_list += ['mitchellbrendan', 'boyle', 'darya', 'moghimi', 'zacher', 'susanne', 'scheibe', 'nico', 'yperen', "tassilo", "momm"]
author_list += ['gerhard', "blickle", 'yongmei', 'andreas', "wihler", 'mareike', "kholin", 'jochen', "menges", 'francesco']
author_list += ['montani','massimo', 'maoret','lucas', 'dufour', 'diego', 'montano', 'anna', 'reeske', 'franziska', 'franke']
author_list += ['joachim', 'hüffmeier', 'frank','ramona', 'bobocel', 'michael','mumfordyitzhak', 'fried', 'timothy','munyonrachel']
author_list += ['kane', 'frieder', 'charles', 'murnieks', 'anthony', 'klotz', 'teresa', 'müllercornelia']
author_list += ['niessen', 'mohamed', 'ikram', 'nasr', 'assâad', 'akremi', 'jacqueline', 'coyle', 'shapiro', "eitan", "naveh",'katz']
author_list += ["navon","stern", 'thomas','nglorenzo', 'lucianetti', 'jessica','nicklinpaul','spector', 'cornelia', 'niessen', 'müller']
author_list += ['sabine', 'hommelhoff', 'mina', 'westman', 'karen', 'nivenluke']
author_list += ['boorman', 'norton', 'stacey', 'neal', 'ashkanasy', 'rikki', 'nouri', 'erez', 'jian', 'liang', 'brendan', 'bannister']
author_list += ['warren', 'chiu', 'kimberly',"brienterry",'beehr', 'jane', "learyjörgen", 'sandberg', 'anna', 'carmella']
author_list += ['ocampo', 'kohyar', 'kiazad', 'jacoba', 'oedzes', 'gerben', 'vegt', 'floor', 'rink', 'walter', 'florian', 'offergelt']
author_list += ['klaus', 'moser', 'shaw','brian', 'holtz','seongsu','sandra', 'ohly','sabine', 'sonnentag','franziska', 'pluntke', 'hakan']
author_list += ['ozcelik', 'youngah', 'parkverena','haun', 'youngah']
author_list += ['parkjustin','sprung', "vesa", "peltokorpi", "allen", 'fabian', "froese","pengdongkyu",'chunyan', 'pengwei', 'zeng']
author_list += ['mark','petersontais', 'siqueira', 'barreto', 'mark', 'edward', 'pickering']
author_list += ['matthew','piszczek', 'pollack', 'ernest', "boyle", 'corinne', 'post', 'roman', 'prem', 'sandra', 'ohly']
author_list += ['kubicek', 'korunka', 'manuela', 'priesemuthregina','taylor', 'alexander', 'pundtlaura', 'venz', 'minya', 'duan']
author_list += ["yilong", "direnzo", 'yilong', "duan", 'jennifer','ragsdaleterry','beehr', 'stefan', 'razinskasmartin']
author_list += ['hoegl', 'young', 'rheejin', 'choi', 'kirsten', 'robertsonjane', "reilly", 'mark','roehlingjason', 'huang']
author_list += ['jade', 'talbot', 'alex', 'rubenstein', 'kammeyer', 'mueller', 'tomas', 'thundiyil', 'jörgen', 'sandbergharidimos']
author_list += ['tsoukas', 'carsten','schermulybertolt', 'meyer', "pauline", "schilpzand", 'irene', "pater", 'amir', "erez"]
author_list += ['benjamin', 'schneider', 'allison', 'yost', 'kropp', 'cory', 'kind', 'holly', 'jeremy','schoen', 'selenko','mäkikangas']
author_list += ['stride','shan','joshua', 'keller','damien', 'joseph', 'ping']
author_list += ['shao','andrew','mary', 'mawritz', "meir", "shemla", 'bertolt', "meyer", 'lindred', "greer", 'shung']
author_list += ['shin','feirong', 'yuan','jing', 'zhou', 'shockley', 'heather', 'ureksoy', 'ozgun', 'burcu', 'rodopman']
author_list += ['poteat', 'ryan', 'dullaghan', 'shoham','almor','smahammad','jost']
author_list += ['siewekebin', 'zhao', 'barjinder', 'singh', 'margaret', 'shaffer', 'selvarajan', 'adam', 'smale', 'silvia', 'bagdadli']
author_list += ['rick', 'cotton', 'dello', 'russo', 'dickmann', 'martina', 'gianecchini', 'kaše', 'mila', 'lazarova', 'reichel', 'rozo']
author_list += ['marijke', 'verbruggen', 'ifedapo', 'adeleye', 'maike', 'andresen', 'eleni', 'apospori', 'olusegun', 'babalola']
author_list += ['briscoe', 'seok', 'katharina', 'chudzikowski', 'nicky', 'dries', 'petra', 'eggenhofer', 'rehart', 'zhangfeng']
author_list += ['martin', 'gubler', 'douglas', 'svetlana', 'khapova', 'najung', 'evgenia', 'lysova', 'sergio', 'madero', 'debbie']
author_list += ['mandel', 'wolfgang', 'mayrhofer', 'biljana', 'bogićević', 'milikić', 'sushanta', 'kumar', 'mishra', 'chikae', 'naito']
author_list += ['emma', 'parry', 'noreen', 'saher', 'richa', 'saxena', 'nanni', 'schleicher', 'florian', 'schramm', 'pamela', 'mami']
author_list += ['taniguchi', 'julie', 'unite']
author_list += ["bryan", "acton","roseanne", "foti", "robert", "lord", "jessica", "gladfelter", "mats"]
author_list += ["alvesson","katja", "einola", 'steina', 'jantonak', "john", "antonakis", "samuel", "bendahan", "philippe"]
author_list += ["jacquart", "rafael", "lalive", "george", "banks", "nicolas", "bastardoz", "michael", "cole", "david","alice"]
author_list += ["eagly", "olga", "epitropaki"]
author_list += ["william", "gardner", 'alexander', "haslam", "hogg", "ronit"]
author_list += ["kark", "kevin", "lowe", "philip", "podsakoff", "seth", "spain", "janka", "stoker", "niels", "quaquebeke"]
author_list += ["mark", "vugt", "dusya", "vera", "roberto", "weber", "nicolas", "bastardoz","mark", "vugt", 'stephane']
author_list += ['brutus', 'shawn', 'burke', 'dana', 'sims', 'elizabeth', 'lazzara', 'eduardo', 'salas', 'abraham', 'carmeli']
author_list += ['meyrav', 'yitzack', 'halevi', "david", "carrington", "combe", "mumford", 'jingnan', 'chen', "minyoung", "cheong"]
author_list += ["francis", "yammarino", "shelley", "dionne", "chou", "tsai",'cheng', "steve","shin", 'guang']
author_list += ["liang", "amon", "chizema","ganna", "pogrebna", 'neil', 'stewart', "joseph", "crawford",'anne', "kelder", "stéphane"]
author_list += ["côté", "paulo", "lopes", "peter", "salovey", "christopher", "miners"]
author_list += ['uqbweave', 'david',"david", "hock", 'peng',"belle", "derks","colette", "laar","naomi", "ellemers", 'shelley', 'dionne']
author_list += ['yvonne', 'budden', 'nathan',"bassam", "farah", "rida"]
author_list += ["elias", "cristine", "clercy", "glenn", "rowe", 'christopher', 'watkins', "louis", "sean", "hannah", "noel"]
author_list += ["fred", "walumbwa", "zachary", "garfield", "rueden", "edward", "hagen", 'geys', 'steffen', 'giessner']
author_list += ['daan', 'knippenberg', 'sleebos', "laura", "giurge", "marius", "dijke", "michelle", "zheng", "cremer", 'design']
author_list += ['unit', 'london', 'school', 'economics','activepdf', 'kelly', 'hannum', "anna", 'luca', "heimann"]
author_list += ["ingold", "martin", "kleinmann", "nathan", "hiller", "hock", 'peng', "ajay", "ponnapalli", "sibel", "ozgen"]
author_list += ['crystal', 'hoyt', 'stefanie', 'johnson', 'susan', 'elaine', 'murphy', 'kerry', 'hogue', 'skinnell', 'hendrik']
author_list += ['huettermann', 'sebastian', 'doering', 'sabine', 'boerner', "hughes", "allan", "tian", "alex", "newman", "alison"]
author_list += ["legood", 'center', 'applied', 'social', 'research', 'dongil', 'jung', 'francis', 'yammarino',"thomas"]
author_list += ["kelemen", "matthews", "kimberley", "breevaart", 'michael', 'kosfeld', "lindie", "liang", "douglas", "brown"]
author_list += ["huiwen", "lian", "hanig", 'lance', "ferris", "lisa", "keeping", "jukka", "lipponen", "janne", "kaltiainen"]
author_list += ["werff", "niklas", "steffens", "jeffrey", "lovelace", "brett", "neely", "julian", "allen", "hunter"]
author_list += ['bourgoin', "charles", "reilly", "bernadette", "doerr", "caldwell", "jennifer", "chatman", "reilly"]
author_list += ['dawn', 'eubanks', "philip", "podsakoff","nathan", "podsakoff", "therese", "reitan","sten"]
author_list += ["stenberg", 'chester', 'schriesheim', 'joshua', 'terri', 'scandura', 'jeroen', 'staff', "maria"]
author_list += ["tims", "arnold", "bakker", "despoina", "xanthopoulou", "chou", "tsai", 'chih', "wang", 'shiuan', "cheng"]
author_list += ["herman","huang","wing",'hardin','kragt','djohnston', 'billings']
author_list += ["fred", "walumbwa", "peng", "john", "schaubroeck", "bruce", "avolio", 'user', "philip", "yang"]
author_list += ["riepe", "katharina", "moser", "kerstin", "pull", "siri", "terjesen", "christian", "zehnder", "holger"]
author_list += ["herz", "jean", 'philippe', "bonardi", 'school', 'business', 'msmhh']
author_list += ['ambika', 'ambika', 'natalie', 'allen', 'tracy', 'hecht', 'neil', 'anderson', 'sarah', 'sleap', 'nikos']
author_list += ['bozionelos', 'adam', 'butler', 'amie', 'skattebo', 'john', 'cordery', 'kevin', 'daniels', 'claire', 'harris']
author_list += ['briner', 'taru', 'feldt', 'mika', 'kivimaumlki', 'anne', 'rantala', 'asko', 'tolvanen', 'donald', 'gardner']
author_list += ['linn', 'dyne', 'pierce', 'filip', 'lievens', 'frederik', 'anseel', 'adam', 'meade', 'james', 'meindl']
author_list += ['paul', 'paulus', 'karen', 'charlie', 'reeve', 'eric', 'heggestad', 'astrid', 'richardsen','zhao','zhou','jing']
author_list += ['monica', 'martinussen', 'anit', 'somech', 'anat', 'drach', 'zahavy', 'gigi', 'sutton', 'mark', 'griffin']
author_list += ['william', 'turnley', 'mark', 'bolino', 'scott', 'lester', 'james', 'bloodgood', 'joan', 'horn', 'toon', 'taris']
author_list += ['wilmar', 'schaufeli', 'paul', 'schreurs', 'fred', 'walumbwa', 'peng', 'wang', 'john', 'lawler']
author_list += ['michael', 'west', 'felix', 'brodbeck', 'andreas', 'richter', 'mark', 'wilson', 'david', 'dejoy']
author_list += ['robert', 'vandenberg', 'hettie', 'richardson', 'allison', 'mcgrath', 'ambika','david','micheal']
author_list += ['elena','belogolovsky','peter','bamberger','karlene','roberts','chris','bingham','campbell','seung','hwan','jeong','scott','graffin']
author_list += ['robert','messen','andrew','carton','brice','roberts','dattée','oliver','alexy','erkko','autio','sreedhari','desai','patricio']
author_list += ['duran','nadine','kammerlander','marc','essen','thomas','zellweger','john','joseph','wshong','gokhan','ertug','tamar','yogev']
author_list += ['yonghoon','lee','peter','hedström','kibler','ewald','markus','perkmann','brenda','flannery','douglas','michael','florin','lubatkin']
author_list += ['william','schulze','raghu','garud','sanjay','jain','arun','kumaraswamy','gerard','george','christopher','corbishley','jane','khayesi']
author_list += ['martine','haas','laszlo','tihanyi','wakenshaw','gibbons','deborah','brian','gunia','joo','hun','han','saehee','kang']
author_list += ['rebecca','kehoe','david','lepak','hausknecht','nathan','hiller','robert','vance','hitt','tina','dacin','edward','levita']
author_list += ['jean','arregle','luc','anca','borza','fmurray','philippe','jacquart','antonakis','jason','jay','fuller','elfenbeinh','cynthia']
author_list += ['kim','jensen','lorraine','crystal','hkpu','margaret','luciano','amy','bartels','lauren','innocenzo','travis','maynard','mathieu']
author_list += ['hardy','smagui','johanna','mair','ignasi','marti','marc','ventresca','dellring','jcarson','mmcardle','erez','mia','mathieu']
author_list += ['monin','niels','noorderhaven','eero','vaara','kroon','jeroen','neckebrouck','gina','dokko','lisa','hisae','nishii','sonja']
author_list += ['opper','donde','ashmos','plowman','lakami','baker','tammy','beck','mukta','kulkarni','stephanie','solansky','deandra','villarreal']
author_list += ['ridge','ingram','aaron','hill','bloy','drew','harry','dwight','lemke','richard','dino','staf','guillaume','soenen']
author_list += ['tessa','melkonian','maureen','ambrose','evans','bennett','tepper','nikolaos','dimotakis','schurer','lambert','joel','koopman','fadel']
author_list += ['matta','hee','man','park','wongun','goo','tepper','varkey','titus','owen','parker','francesca','gino','bass']
author_list += ['erin','charlie','trevor','home','computer','martins','martín','esmt','european','varkey','gmbh','cwbauman','school','francesca','gino','bass']

In [8]:
# Function to filter out the stopwords and authors names
def filterWords(msg, words):
    m = msg.split(' ')
    words_end = [w + '.' for w in words]
    filtered_words = list(filter(lambda word: word not in words, m))
    filtered_words = [(w if w not in words_end else '.') for w in filtered_words]
    result = ' '.join(filtered_words)
    return(result)

In [9]:
data_proc_auth = [filterWords(f, author_list) for f in pdf_p] #running

In [10]:
data_proc_auth[2]

'. . february . reprints permissions sagepub journalspermissions closer look personality turnover relationship criterion expansion dark traits time purdue national purdue national recent advances personality turnover literatures suggest importance expanding current turnover criteria incorporating dark personality traits examining role time relationships.present study investigates issues considering speed reasons leaving examining wider range personality variables predictors including bright dark traits exploring potential moderating effect time predictions.data collected sample employees working manufacturing .using bayesian survival analysis framework found dark traits useful predicting turnover outcomes traditional personality traits predicted specific turnover reasons deviant call show investigating role time showed satisfaction intellectual curiosity grew predictive strength course organizational tenure time dependent effects predictors negligible.keywords personality turnover baye

In [11]:
stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']
stopwords += ['january', 'february', 'march', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
# author name to remove
stopwords +=['karina nielsen','kevin daniels','elaine','berlin','catherine', 'jaewan yang','andrew O herdman','amanda', 'sabine','kerstin','kertin','sagepub','journalspermissions','catheine','chidiebere ogbonnaya','john wiley','sons ltd','martin','jeffrey B arthur']

In [12]:
data_proc = [filterWords(f, stopwords) for f in data_proc_auth] #running

In [13]:
data_proc[22]

'. author reprints permissions meta analysis interactive additive relative effects cognitive ability motivation performance tested longstanding belief performance function interaction cognitive ability motivation.using data values obtained primary study authors input used meta analysis assess strength consistency multiplicative effects ability motivation performance.triangulation evidence based types analyses revealed effects ability motivation performance additive multiplicative.additive effects ability motivation accounted explained variance performance ability motivation interaction accounted explained variance addition interaction consistently reflect predicted form findings acknowledgments article accepted editorship previous version paper presented meetings academy management thank brian anonymous reviewers useful constructive feedback allowed improve manuscript substantially thank guidance range restriction corrections.addition thank fred oswald help regression analysis.finally 

# Get country name

In [13]:
import pycountry

country_list = []
for country in pycountry.countries:
    a = (country.name).lower()
    country_list.append(a) 

In [14]:
data_proc_country =[]
for j in data_proc:
    line = j
    for k in country_list:
        line = re.sub(k, '', line)
        
    data_proc_country.append(line)

In [16]:
# save data
pickle.dump(data_proc_country, open( "C:/Thesis/Data/save/journal_of_management/metadata/removed_country/metadata_all.p", "wb" ) )

In [17]:
data_proc_2 = [re.sub(' \.+', ' ', f) for f in data_proc_country]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]
data_proc_2 = [re.sub('http', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('https', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('www', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('iweb', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('_', ' ', f) for f in data_proc_2]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]


In [18]:
data_proc_2[2]

'. reprints permissions closer look personality turnover relationship criterion expansion dark traits time purdue national purdue national recent advances personality turnover literatures suggest importance expanding current turnover criteria incorporating dark personality traits examining role time relationships.present study investigates issues considering speed reasons leaving examining wider range personality variables predictors including bright dark traits exploring potential moderating effect time predictions.data collected sample employees working manufacturing using bayesian survival analysis framework dark traits useful predicting turnover outcomes traditional personality traits predicted specific turnover reasons deviant investigating role time showed satisfaction intellectual curiosity grew predictive strength course organizational tenure time dependent effects predictors negligible.keywords personality turnover bayesian methods survival analysis supplemental material artic

In [19]:
# save data
pickle.dump(data_proc_2, open( "C:/Thesis/Data/save/Master_Data/MD_3/data_proc_latest/JM_data_proc.p", "wb" ) )

In [20]:
#import io
with open("C:/Thesis/Data/save/Master_Data/MD_3/data_proc_latest/JM_data_proc_txt.txt", "w",encoding="utf-8") as outfile:
    for i in range(len(data_proc_2)):
        outstring = ""
        outstring += str(data_proc_2[i])
        outfile.write(outstring)